Aim: To clean all the data that has been scraped into a uniform table before merging them into a larger DataFrame

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# from math import radians, sin, cos, axsin, sqrt
from sklearn.metrics import confusion_matrix, roc_curve, auc
import numpy as np
import patsy

from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict,StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import r2_score,mean_squared_error
from statsmodels.formula.api import ols
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import math
import re

%matplotlib inline
pd.set_option('display.max_columns',500)
pd.options.display.max_colwidth = 400

In [2]:
def check_null(df):
    nulls = df.isnull().sum().sum()
    print("Total NaN:",nulls)

def check_shape(df):
    rows = df.shape[0]
    columns = df.shape[1]
    print("Rows:",rows)
    print("Columns:", columns)
    
def describe(df):
    description= df.describe()
    print(description)

# def drop_unknowncolumn(df):
#     df.drop(['Unnamed: 0'], axis=1, inplace=True)

# def remove_duplicates(df):
#     before_d = len(df)
#     df.drop_duplicates(inplace=True)
#     after_d = len(df)
#     print(str(before_d-after_d) + " duplicates were removed!")

# def reset_index(df):
#     df = df.reset_index(inplace=True)

def eda(df):
    check_null(df)
    check_shape(df)
    describe(df)
#     drop_unknowncolumn(df)

## ASOS Data

In [16]:
asos_skirts = pd.read_csv('../datasets/asos_skirt.csv')
asos_activewear = pd.read_csv('../datasets/asos_activewear.csv')
asos_hoodies = pd.read_csv('../datasets/asos_hoodies.csv')
asos_jumper = pd.read_csv('../datasets/asos_jumper.csv')
asos_loungewear = pd.read_csv('../datasets/asos_loungewear.csv')
asos_shorts = pd.read_csv('../datasets/asos_shorts.csv')
asos_tops = pd.read_csv('../datasets/asos_tops.csv')


In [17]:
all_asos = [asos_skirts, asos_activewear, asos_hoodies, asos_jumper, asos_loungewear, asos_shorts, asos_tops]


In [18]:
for df in all_asos:
    print(df.dtypes)

web-scraper-order         object
web-scraper-start-url     object
link                      object
link-href                 object
name                      object
price                     object
colour                    object
description               object
product code             float64
about me                  object
brand                     object
category                  object
dtype: object
web-scraper-order         object
web-scraper-start-url     object
link                      object
link-href                 object
name                      object
price                     object
colour                    object
description               object
product code             float64
about me                  object
brand                     object
category                  object
dtype: object
web-scraper-order         object
web-scraper-start-url     object
link                      object
link-href                 object
name                      object
price          

In [19]:
for df in all_asos:
    df = df.drop(columns=['web-scraper-order','web-scraper-start-url','link','link-href','product code'],inplace=True)

In [20]:
for row in range(asos_shorts.shape[0]):
    asos_shorts.loc[row,'category']= 'shorts'
for row in range(asos_skirts.shape[0]):
    asos_skirts.loc[row,'category']= 'skirts'
for row in range(asos_hoodies.shape[0]):
    asos_hoodies.loc[row,'category']= 'outerwear'
for row in range(asos_jumper.shape[0]):
    asos_jumper.loc[row,'category']= 'outerwear'
for row in range(asos_tops.shape[0]):
    asos_tops.loc[row,'category']= 'tops'
# for row in range(asos_activewear.shape[0]):
#     asos_activewear.loc[row,'category']= 'activewear'
for row in range(asos_loungewear.shape[0]):
    asos_loungewear.loc[row,'category']= 'loungewear'

In [21]:
asos_activewear.category.unique()

array(['T-shirt', 'Leggings', nan, 'bra', 'Vest', 'Plus-size top',
       'Jacket', 'Top', 'Plus-size shorts', 'Trainers', 'Socks',
       'Protest', 'Sweatpants', 'Hoodie', 'Jumpsuit', 'Shorts',
       'Tall leggings', 'Petite leggings', 'Sweatshirt', 'Scarf',
       'Jumper', 'Ski salopette', 'Swimsuit', 'shorts', 'The North Face',
       'Hat', 'Trousers', 'vest', 'Ski trousers', 'Bikini top', 'Bag',
       'Bum bag', 'Body', 'Maternity leggings', 'ASOS 4505', 'sweatpants',
       'Pullover', 'Maternity vest', 'Gloves', 'Watch', 'Coat',
       'Tall top', 'Maternity top', 'Nike', 'Bra', 'Bikini', 'Fitbit',
       'Plus-size trousers/leggings', 'Plus-size leggings', 'Dungarees',
       'Maternity trousers', 'Boots', 'leggings', 'Tall shorts',
       'Lipgloss', 'Ultrasun', 'Rip Curl', 'Roxy', 'Petite jumper', 'top',
       'Shirt'], dtype=object)

In [22]:
asos_activewear.category = asos_activewear.category.str.lower()
asos_activewear['category']=asos_activewear['category'].astype(str)

In [23]:
for row in range(asos_activewear.shape[0]):
    if "top" in asos_activewear['category'][row]:
        asos_activewear.loc[row,'category'] = 'tops'
    elif "leggings" in asos_activewear.category[row]:
        asos_activewear.loc[row,'category'] = 'bottoms'
    elif "t-shirt" in asos_activewear.category[row]:
        asos_activewear.loc[row,'category'] = 'top'
    elif "short" in asos_activewear.category[row]:
        asos_activewear.loc[row,'category'] = 'shorts'
    elif "pants" in asos_activewear.category[row]:
        asos_activewear.loc[row,'category'] = 'bottoms'
    elif "hoodie" in asos_activewear.category[row]:
        asos_activewear.loc[row,'category'] = 'outerwear'
    elif "vest" in asos_activewear.category[row]:
        asos_activewear.loc[row,'category'] = 'outerwear'
    elif "sweatshirt" in asos_activewear.category[row]:
        asos_activewear.loc[row,'category'] = 'outerwear'
    elif "coat" in asos_activewear.category[row]:
        asos_activewear.loc[row,'category'] = 'outerwear'
    elif "jacket" in asos_activewear.category[row]:
        asos_activewear.loc[row,'category'] = 'outerwear'
    elif "bra" in asos_activewear.category[row]:
        asos_activewear.loc[row,'category'] = 'lingerie'
    elif "jumpsuit" in asos_activewear.category[row]:
        asos_activewear.loc[row,'category'] = 'jumpsuits'
    elif "jumper" in asos_activewear.category[row]:
        asos_activewear.loc[row,'category'] = 'outerwear'
    elif "trouser" in asos_activewear.category[row]:
        asos_activewear.loc[row,'category'] = 'bottoms'
    elif "shirt" in asos_activewear.category[row]:
        asos_activewear.loc[row,'category'] = 'tops'
        

In [24]:
categories = ['tops','dress','skirts','outerwear','bottoms','jumpsuits','shorts','jeans']
for row in range(asos_activewear.shape[0]):
        if asos_activewear['category'][row] not in categories:
            asos_activewear.drop(row,inplace=True)

In [25]:
def split_it(string):
    return re.sub(r"([a-z])([A-Z])", r"\1 \2", string)
for df in all_asos:
    #clean the prices
    df['price'] = df['price'].astype(str)
    df['price']=[x[4:] for x in df.price]
    # change titles
    df_newcols = {
        'about me':'about_me'
    }
    df.rename(columns=df_newcols,inplace=True)
    
    #clean description
    df['about_me'] = df['about_me'].astype(str)
    df['about_me']=[x[10:] for x in df.about_me]

#     #splits words where there is one capital and lowercase together
    df['about_me'] = df['about_me'].apply(split_it)
    df['description'] = df['description'].astype(str)
    df['description']= df['description'].apply(split_it)

    #remove \n and replace with a space
    df['about_me']= df['about_me'].str.replace('\n',' ')

In [26]:
asos_shorts.head()

,name,price,colour,description,about_me,brand,category
0,ASOS DESIGN co-ord mesh legging short with crystal studs,37.05,Pink,Part of a co-ord set Top sold separately High rise Stretch waistband Longline length Bodycon fit A tight cut to the body,"Lined mesh All-over embellishment Main: 95% Nylon, 5% Elastane.",ASOS DESIGN,shorts
1,Warehouse tailored shorts with belt in black,53.73,Black,For your daytime thing High-rise elasticated waistband Removable tie belt Zip fly with hook-and-bar fastening Side pockets Regular fit Just select your usual size,"Thick woven fabric Unlined and stretchy Main: 97% Cotton, 3% Elastane.",Warehouse,shorts
2,Nike Cream High Waist Runner Shorts,50.02,Pale ivory/(black),For your daytime thing High rise Drawstring waistband Branded design Curved hem Regular fit Just select your usual size,Soft sweatshirt fabric A great chill-out partner Main: 100% Cotton.,Nike,shorts
3,Nike Training flex 2 in 1 shorts in black,46.32,Black,"For when you need a bit more motivation Double-layer design with stretch shorts lining Internal drawstring waist Nike Swoosh logo Regular fit No surprises, just a classic cut","Smooth, lightweight fabric Uses Nike Dri-FIT technology Helps move sweat away from the skin Sweat is then drawn out of the fabric to speed up evaporation Keeping you cool and dry Body: 88% Polyester, 12% Spandex.",Nike,shorts
4,ASOS DESIGN Tall sweat short,22.23,Grey marl,Add-to-bag potential: considerable High rise Elasticated waist Side pockets Regular fit Just select your usual size,"Soft sweatshirt fabric Style and comfort – it's a lifestyle Main: 70% Polyester, 30% Cotton.",ASOS DESIGN,shorts


In [27]:
for df in all_asos:
    print(df.isnull().sum())

name            0
price           0
colour         16
description     0
about_me        0
brand          64
category        0
dtype: int64
name           0
price          0
colour         3
description    0
about_me       0
brand          4
category       0
dtype: int64
name            0
price           0
colour         26
description     0
about_me        0
brand          76
category        0
dtype: int64
name            1
price           0
colour         72
description     0
about_me        0
brand          20
category        0
dtype: int64
name             0
price            0
colour           0
description      0
about_me         0
brand          133
category         0
dtype: int64
name            0
price           0
colour         68
description     0
about_me        0
brand          30
category        0
dtype: int64
name             1
price            0
colour          37
description      0
about_me         0
brand          478
category         0
dtype: int64


In [28]:
for df in all_asos:
    #drop all null name columns
    null_columns=df.columns[df.isnull().any()]
    df[null_columns].isnull().sum()
    drop=list((df[df["name"].isnull()][null_columns]).index)
    df = df.drop(drop,inplace=True)
    

In [29]:
def get_brand(string):
    string = string.split()
    words = [word for word in string if word[0].isupper()]
    return ' '.join(word for word in words)
for df in all_asos:
    #extract brand name from name
    df['brand'].fillna(df['name'].apply(get_brand), inplace=True)
    #fill empty colour columns with the word 'missing'
    df['colour'].fillna('missing', inplace=True)

In [30]:
# # all_asos = [asos_skirts, asos_activewear, asos_hoodies, asos_jumper, asos_loungewear, asos_shorts, asos_tops]
# #fill all empty category column with category names based on category it was scraped from
# asos_skirts['category'].fillna('skirts',inplace=True)
# asos_activewear['category'].fillna('activewear',inplace=True)
# asos_hoodies['category'].fillna('hoodies',inplace=True)
# asos_jumper['category'].fillna('jumper',inplace=True)
# asos_loungewear['category'].fillna('loungewear',inplace=True)
# asos_shorts['category'].fillna('shorts',inplace=True)
# asos_tops['category'].fillna('tops',inplace=True)




In [31]:
asos_loungewear.head(20)

,name,price,colour,description,about_me,brand,category
0,ASOS DESIGN lounge zip sweat & short set,51.88,Black,Part of our responsible edit Half-zip neck Dropped shoulders Long sleeves Short bottoms Stretch waist Regular fit Just select your usual size,"Soft sweatshirt fabric We partner with the Better Cotton Initiative to improve cotton farming globally This makes it better for farmers and the environment BCI provides farming-practice training It promotes things like water efficiency and reducing the most harmful chemicals Body: 70% Cotton, 30% Polyester.",ASOS DESIGN,loungewear
1,Monki eyes print pyjama set in black,46.32,Black,Part of our responsible edit All-over eye print Crew neck Short sleeves Trouser bottoms Regular fit Just select your usual size,"Soft jersey Made with organic cotton Grown with less water and no pesticides It's a win-win, improving the environment for both wildlife and cotton farmers Main: 95% Organic Cotton, 5% Elastane.",Monki,loungewear
2,Lasula lounge crop top with contrast trim co ord in black,29.64,Black,Part of a co-ord set Bottoms sold separately Round neck Short sleeves Printed underband Cropped length Regular fit Just select your usual size,"Stretch jersey A soft, versatile fabric Fabric: 100% Polyester.",Lasula,loungewear
3,ASOS DESIGN Maternity mix & match pyjama jersey tee and trouser,14.82,Coral,nan,,ASOS DESIGN Maternity,loungewear
4,ASOS DESIGN Maternity lounge waffle sweat,33.35,Grey,Your downtime uniform Round neck Plain design Regular fit Designed to fit you from bump to baby,"Waffle-textured fabric Only the same in texture, not taste We could go for a waffle, though Body: 65% Cotton, 35% Polyester.",ASOS DESIGN,loungewear
5,ASOS DESIGN Curve botanical tiger shirt & short 100% modal pyjama,48.17,Navy,Part of our responsible edit Tiger print Revere collar Chest pocket Button placket Matching shorts Regular fit Just select your usual size,"Lightweight, soft modal It's made from wood pulp from responsibly managed forests It lowers environmental impact by using less energy Main: 100% Modal.",ASOS DESIGN Curve,loungewear
6,ASOS DESIGN Maternity hoodie & jogger set,44.46,Black,Your downtime uniform Drawstring hood Pouch pocket Trouser bottoms Drawstring waistband Fitted cuffs Regular fit True to size Designed to fit you from bump to baby,"Lightweight jersey For when comfort is key Body: 64% Polyester, 36% Cotton.",ASOS DESIGN,loungewear
7,ASOS DESIGN Curve satin button front cami lace short set,48.17,Black,Add-to-bag potential: considerable V-neck Button placket Adjustable straps Lace panel Short bottoms Stretch waist Regular fit Just select your usual size,"Satin-style fabric Silky and super smooth Main: 100% Polyester, Trim: 100% Nylon.",ASOS DESIGN Curve,loungewear
8,ASOS DESIGN co-ord off shoulder cable jumper in super fluffy yarn,51.88,Camel,Part of a co-ord set Bottoms sold separately Boat neck Cable pattern Ribbed cuffs and hem Relaxed fit Think baggy and casual,"Lightweight, fluffy knit For a textural switch-up Body: 100% Polyester.",ASOS DESIGN,loungewear
9,ASOS DESIGN Tall tracksuit cute sweat / basic jogger with tie with contrast binding,59.29,Sand,"Part of our responsible edit Tracksuit set Your casual vibe, sorted Plain sweatshirt Crew neck Over-the-head style Contrast piping Ribbed trims Trouser bottoms Drawstring waistband Regular fit Just select your usual size","Soft sweatshirt fabric We partner with the Better Cotton Initiative to improve cotton farming globally This makes it better for farmers and the environment BCI provides farming-practice training It promotes things like water efficiency and reducing the most harmful chemicals Fabric 1: 100% Cotton, Fabric 2: 97% Cotton, 3% Elastane.",ASOS DESIGN Tall,loungewear


In [32]:
for df in all_asos:
    print(df.isnull().sum())

name           0
price          0
colour         0
description    0
about_me       0
brand          0
category       0
dtype: int64
name           0
price          0
colour         0
description    0
about_me       0
brand          0
category       0
dtype: int64
name           0
price          0
colour         0
description    0
about_me       0
brand          0
category       0
dtype: int64
name           0
price          0
colour         0
description    0
about_me       0
brand          0
category       0
dtype: int64
name           0
price          0
colour         0
description    0
about_me       0
brand          0
category       0
dtype: int64
name           0
price          0
colour         0
description    0
about_me       0
brand          0
category       0
dtype: int64
name           0
price          0
colour         0
description    0
about_me       0
brand          0
category       0
dtype: int64


In [33]:
for df in all_asos:
    df['price'] = pd.to_numeric(df['price'], errors='coerce').fillna(0)

In [34]:
# getting rid of all rows where price is zero
# all_asos = [asos_skirts, asos_activewear, asos_hoodies, asos_jumper, asos_loungewear, asos_shorts, asos_tops]
asos_shorts= asos_shorts[asos_shorts['price']>0]
asos_skirts= asos_skirts[asos_skirts['price']>0]
asos_activewear= asos_activewear[asos_activewear['price']>0]
asos_hoodies= asos_hoodies[asos_hoodies['price']>0]
asos_jumper= asos_jumper[asos_jumper['price']>0]
asos_loungewear= asos_loungewear[asos_loungewear['price']>0]
asos_tops= asos_tops[asos_tops['price']>0]

In [35]:
for df in all_asos:
    print(df.shape)

(1720, 7)
(560, 7)
(819, 7)
(1541, 7)
(435, 7)
(530, 7)
(6472, 7)


In [36]:
all_asos = [asos_skirts, asos_activewear, asos_hoodies, asos_jumper, asos_loungewear, asos_shorts, asos_tops]
asos = pd.concat(all_asos)
asos = asos.reset_index(drop=True)


In [37]:
asos.isnull().sum()

name           0
price          0
colour         0
description    0
about_me       0
brand          0
category       0
dtype: int64

In [38]:
all_asos = [asos_skirts, asos_activewear, asos_hoodies, asos_jumper, asos_loungewear, asos_shorts, asos_tops]

for a in all_asos:
    print (a.category.unique())


['skirts']
['bottoms' 'outerwear' 'tops' 'shorts' 'jumpsuits']
['outerwear']
['outerwear']
['loungewear']
['shorts']
['tops']


In [39]:
asos.to_csv("asos.csv")

In [40]:
asos.head()

,name,price,colour,description,about_me,brand,category
0,& Other Stories velvet jacquard ruched mini skirt in black,101.90,Black,Coming soon to your Saved Items Stretch waistband Ruched design Ruffled hem Bodycon fit A tight cut to the body,"Velvet jacquard Drapey fabric with an intricate, interwoven pattern Main: 95% Polyester, 5% Elastane.",& Other Stories,skirts
1,ASOS DESIGN neon boucle suit skirt,55.58,Check,For your daytime thing Check design High-rise waist Zip-side fastening Slim fit Cut close to the body,"Textured boucléLooped fabric with a woolly finish Body: 68% Acrylic, 25% Polyester, 7% Viscose, Lining: 100% Polyester.",ASOS DESIGN,skirts
2,Liquorish pleated midaxi skirt in pink neon colour block,101.90,Pink,For that thing you have to go to Pleated design High rise Stretch waistband Midi length Regular fit Just select your usual size,Smooth woven fabric It's light and floaty Fabric: 100% Polyester.,Liquorish,skirts
3,Vila faux leather skirt,55.58,Black,Add-to-bag potential: considerable High-rise waist Kick split Zip-back fastening Regular fit Just select your usual size,"Faux leather Looks like real leather, but it's not Pointing out the obvious Body: 100% Polyester.",Vila,skirts
4,People Tree velvet mini skirt,120.43,Black,Easily dressed up or down High-rise waist Button detail Regular fit Just select your usual size,Soft velvet One smooth operator Body: 100% Cotton.,People Tree,skirts


## Zalora

In [81]:
#omitted tops
zal_jumpsuits = pd.read_csv("../datasets/zalora_jumpsuits.csv")
zal_pants = pd.read_csv("../datasets/zalora_pantsleggings.csv")
zal_skirts = pd.read_csv("../datasets/zalora_skirts.csv")
zal_tops = pd.read_csv("../datasets/zalora_tops.csv")
zal_dresses = pd.read_csv("../datasets/zalora_dresses.csv")
zal_shorts = pd.read_csv("../datasets/zalora_shorts.csv")
zal_jackets = pd.read_csv("../datasets/zalora_jackets.csv")
zal_blazers = pd.read_csv("../datasets/zalora_blazers.csv")
zal_jeans = pd.read_csv("../datasets/zalora_jeans.csv")
zal_cardi = pd.read_csv("../datasets/zalora_cardi.csv")
zal_hoodies = pd.read_csv("../datasets/zalora_hoodies.csv")
zal_petite = pd.read_csv("../datasets/zalora_petite.csv")
# zal_lingerie = pd.read_csv("../datasets/zalora_lingerie.csv")

In [82]:
all_zalora = [zal_jumpsuits,zal_pants,zal_skirts,zal_tops,zal_dresses,
             zal_shorts,zal_jackets,zal_blazers,zal_jeans, zal_cardi, zal_hoodies]
for df in all_zalora:
    print(df.columns)
    

Index(['Unnamed: 0', 'url', 'brand', 'title', 'price', 'description', 'colour',
       'material'],
      dtype='object')
Index(['Unnamed: 0', 'url', 'brand', 'title', 'price', 'description', 'colour',
       'material'],
      dtype='object')
Index(['Unnamed: 0', 'brand', 'title', 'price', 'description', 'colour',
       'material'],
      dtype='object')
Index(['web-scraper-order', 'web-scraper-start-url', 'link', 'link-href',
       'title', 'brand', 'price', 'description', 'material', 'colour',
       'category'],
      dtype='object')
Index(['Unnamed: 0', 'brand', 'title', 'price', 'description', 'colour',
       'material'],
      dtype='object')
Index(['Unnamed: 0', 'brand', 'title', 'price', 'description', 'colour',
       'material'],
      dtype='object')
Index(['Unnamed: 0', 'brand', 'title', 'price', 'description', 'colour',
       'material'],
      dtype='object')
Index(['Unnamed: 0', 'brand', 'title', 'price', 'description', 'colour',
       'material'],
      dtype='obj

In [83]:
zal_tops.head()

,web-scraper-order,web-scraper-start-url,link,link-href,title,brand,price,description,material,colour,category
0,1573746651-12603,https://www.zalora.sg/women/clothing/fashion-tops/?from=header,55% OFFICHIFlorence TeeS$ 70.90NOW S$ 31.90International Sizes:XSSML,https://www.zalora.sg/ichi-florence-tee-black-1226899.html,Florence Tee,ICHI,S$ 70.90,- Heart motif casual tee\n- V-neckline\n- Unlined\n- Regular fit\n- Modal blend\n- Model wears an S and is 175cm tall,"67% Modal, 33% Polyester",Black,Tops
1,1573743666-11755,https://www.zalora.sg/women/clothing/fashion-tops/?from=header,11% OFFNikeNike Dri-Fit TopS$ 45.00NOW S$ 39.90International Sizes:L,https://www.zalora.sg/nike-nike-dri-fit-top-pink-988238.html,Nike Dri-Fit Top,Nike,S$ 45.00,- Best for training\n- Two toned Dri-Fit tank top\n- Scoop neckline\n- Unlined\n- Regular fit\n- Dri-Fit technology\n- Polyester\n- Model wears an S and is 180cm tall,Polyester,Pink Rise/Heather/White/White,Tops
2,1573761740-16199,https://www.zalora.sg/women/clothing/fashion-tops/?from=header,8% OFFFOREVER 21Buffalo Plaid Crop TopS$ 36.90NOW S$ 33.90International Sizes:L,https://www.zalora.sg/forever-21-buffalo-plaid-crop-top-black-1230734.html,Buffalo Plaid Crop Top,FOREVER 21,S$ 36.90,- Checkered print puff sleeve crop top\n- Square neckline\n- Slim fit\n- Polyblend,Polyblend,Black/Ivory,Tops
3,1573777597-18132,https://www.zalora.sg/women/clothing/fashion-tops/?from=header,52% OFFBoohooWoven Button Back Shell TopS$ 32.90NOW S$ 15.90UK Sizes:68101214,https://www.zalora.sg/boohoo-woven-button-back-shell-top-red-1115193.html,Woven Button Back Shell Top,Boohoo,S$ 32.90,- Crepe top with ruffle trim\n- Round neckline\n- Regular fit\n- Back button fastening\n- Short ruffled sleeves\n- Hem with frill\n- Polyester blend\n- Model wears a UK 8 and is 174cm tall,"97% Polyester, 3% Elastane",Red,Tops
4,1573756032-14999,https://www.zalora.sg/women/clothing/fashion-tops/?from=header,10% OFFMama.liciousMaternity Patricia Long Sleeve Mix TopS$ 59.00NOW S$ 52.90International Sizes:SMLXL,https://www.zalora.sg/mama-licious-maternity-patricia-long-sleeve-mix-top-black-1262853.html,Maternity Patricia Long Sleeve Mix Top,Mama.licious,S$ 59.00,- Solid tone sheer plumeti sleeve top\n- Round neckline\n- Unlined\n- Regular fit\n- Back teardrop button fastening\n- Viscose blend\n- Model wears an S and is 173cm tall,"95% Viscose, 5% Elastane",Black,NaN


In [84]:
# remove_columns = [zal_jumpsuits, zal_pants, zal_skirts, zal_dresses, zal_shorts, zal_jackets, zal_blazers, zal_jeans, zal_cardi, zal_hoodies, zal_petite]
features = ['brand','title','price','description','colour']
# def choose_features(df):
#     df = df[features]

# choose_features(zal_jumpsuits)

In [85]:
zal_jumpsuits = zal_jumpsuits[features]
zal_pants = zal_pants[features]
zal_skirts = zal_skirts[features]
zal_dresses = zal_dresses[features]
zal_shorts = zal_shorts[features]
zal_jackets = zal_jackets[features]
zal_blazers = zal_blazers[features]
zal_jeans = zal_jeans[features]
zal_cardi = zal_cardi[features]
zal_hoodies = zal_hoodies[features]
zal_petite = zal_petite[features]
zal_tops=zal_tops[features]
zal_lingerie=zal_lingerie[features]



In [86]:
zal_lingerie.head()

,brand,title,price,description,colour
0,Cosabella,Never Say Never Padded Racerback Bra,97.9,Monochrome bra with lace detail Regular fit Elasticized underbust Light padded cups Polyamide blend,Black
1,GAP,GapBody Terry Pullover Top,63.9,Graphic drawstring hem long sleeves pajama top Round neckline Relaxed fit Drawstring fastening Modal blend,Grey Lightning Bolt
2,Cosabella,Forza Minikini Panties,47.9,Solid tone sheer mesh lace panties High rise Unlined Slim fit Elasticised waistband Cotton lined gusset Polyamide blend,Wood Rose
3,LYCKA,LYCKA-LMM1026 Lady Sexy Lace Bra And Panty Set-Red,43.1,1/2 cup lace Wired bra Fixed Double Shoulder Strap Back Hook Fastening Sexy and Regular Comfort,Red
4,Kaleidoscope,NALA Classy High Leg Bodysuit,121.0,"NALA Classy High Leg Bodysuit Delicated unique floral motifs at front, removable halter straps offer convertible styling, cut out at sides with strappy back detailing plunging with supportive underwire, adjustable shoulder straps, hook and eye closure at back, lined gusset with snap closure.nderwire, adjustable shoulder straps, hook and eye closure at back, lined gusset with snap clousure.",黑色


In [87]:
#add category
zal_jumpsuits['category'] = 'jumpsuits'
zal_pants['category'] = 'pants'
zal_skirts['category'] = 'skirts'
zal_dresses['category'] = 'dresses'
zal_shorts['category'] = 'shorts'
zal_jackets['category'] = 'jackets'
zal_blazers['category'] = 'blazers'
zal_cardi['category'] = 'cardigans'
zal_hoodies['category'] = 'hoodies'
zal_jeans['category']='jeans'
zal_tops['category']= 'tops'
zal_lingerie['category']='lingerie'

/Users/lynn/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/lynn/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/lynn/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [88]:
zal_pants.head()

,brand,title,price,description,colour,category
0,[],[],[],[],[],pants
1,[],[],[],[],[],pants
2,Banana Republic,Sloan Pants,S$ 121.90,- Solid shade skinny pants\n- Mid rise\n- Skinny fit\n- Front zip and hook fastening\n- Cotton blend,Navy,pants
3,Reebok,Classics Foundation Gigi X Leggings,S$ 49.00,- In collaboration with Gigi Hadid\n- Best for lifestyle\n- Solid tone leggings with Gigi Hadid brandings\n- Unlined\n- High rise\n- Skinny fit\n- Elastic waistband\n- Cotton blend,Black,pants
4,Reebok,Reebok Training Essential Fleece Sweatpants,S$ 75.00,- Best for training\n- Logo printed sweatpants with ribbed cuff locks\n- Unlined\n- Mid rise\n- Slim tapered fit\n- Elastic waistband\n- Drawstring fastening\n- 2 side pockets\n- Cotton blend\n- Model wears an S and is 177cm tall,Black,pants


In [94]:
all_zalora = [zal_jumpsuits, zal_pants, zal_skirts, zal_dresses, zal_shorts, zal_jackets, zal_blazers, zal_jeans, zal_cardi, zal_hoodies, zal_tops]
for df in all_zalora:
    #clean the prices
    df['price'] = df['price'].astype(str)
    df['price']=[x[3:] for x in df.price]

    #remove \n and replace with a space
    df['description'] = df['description'].astype(str)
    df['description']= df['description'].str.replace('\n',' ')
    #splits words where there is one capital and lowercase together

    df['description']= df['description'].apply(split_it)
    df['description']= df['description'].str.replace('-',' ')
    
    for row in range(df.shape[0]):
        df.loc[row, 'description'] = df.loc[row,'description'].split("Model")[0]

In [95]:
zal_pants.head()

,brand,title,price,description,colour,category
0,[],[],,[],[],pants
1,[],[],,[],[],pants
2,Banana Republic,Sloan Pants,121.90,Solid shade skinny pants Mid rise Skinny fit Front zip and hook fastening Cotton blend,Navy,pants
3,Reebok,Classics Foundation Gigi X Leggings,49.00,In collaboration with Gigi Hadid Best for lifestyle Solid tone leggings with Gigi Hadid brandings Unlined High rise Skinny fit Elastic waistband Cotton blend,Black,pants
4,Reebok,Reebok Training Essential Fleece Sweatpants,75.00,Best for training Logo printed sweatpants with ribbed cuff locks Unlined Mid rise Slim tapered fit Elastic waistband Drawstring fastening 2 side pockets Cotton blend,Black,pants


In [96]:
zal_tops.head()

,brand,title,price,description,colour,category
0,ICHI,Florence Tee,70.90,Heart motif casual tee V neckline Unlined Regular fit Modal blend,Black,tops
1,Nike,Nike Dri-Fit Top,45.00,Best for training Two toned Dri Fit tank top Scoop neckline Unlined Regular fit Dri Fit technology Polyester,Pink Rise/Heather/White/White,tops
2,FOREVER 21,Buffalo Plaid Crop Top,36.90,Checkered print puff sleeve crop top Square neckline Slim fit Polyblend,Black/Ivory,tops
3,Boohoo,Woven Button Back Shell Top,32.90,Crepe top with ruffle trim Round neckline Regular fit Back button fastening Short ruffled sleeves Hem with frill Polyester blend,Red,tops
4,Mama.licious,Maternity Patricia Long Sleeve Mix Top,59.00,Solid tone sheer plumeti sleeve top Round neckline Unlined Regular fit Back teardrop button fastening Viscose blend,Black,tops


In [97]:
for df in all_zalora:
    df['price'] = pd.to_numeric(df['price'], errors='coerce').fillna(0)
    df = df.drop(df[df['price']==0 ].index, inplace = True) 
    

In [98]:
zal_pants.head()

,brand,title,price,description,colour,category
2,Banana Republic,Sloan Pants,121.90,Solid shade skinny pants Mid rise Skinny fit Front zip and hook fastening Cotton blend,Navy,pants
3,Reebok,Classics Foundation Gigi X Leggings,49.00,In collaboration with Gigi Hadid Best for lifestyle Solid tone leggings with Gigi Hadid brandings Unlined High rise Skinny fit Elastic waistband Cotton blend,Black,pants
4,Reebok,Reebok Training Essential Fleece Sweatpants,75.00,Best for training Logo printed sweatpants with ribbed cuff locks Unlined Mid rise Slim tapered fit Elastic waistband Drawstring fastening 2 side pockets Cotton blend,Black,pants
5,L'urv,A-Game Logo 7/8 Leggings,101.90,Monochrome toned legging with brand details Mid rise Skinny fit Elasticised waistband Nylon blend,Black,pants
6,Lorna Jane,New Amy 7/8 Tight,110.99,Monochrome tights High rise Slim fit Elasticised waist 1 zip pocket Moisture wicking Polyblend,Black,pants


In [99]:
def reset_index(df):
    df = df.reset_index(inplace=True)
    
for df in all_zalora:
    reset_index(df)
    df=df.drop(columns=['index'],inplace=True)
    

In [100]:
zal_tops.head()

,brand,title,price,description,colour,category
0,ICHI,Florence Tee,70.9,Heart motif casual tee V neckline Unlined Regular fit Modal blend,Black,tops
1,Nike,Nike Dri-Fit Top,45.0,Best for training Two toned Dri Fit tank top Scoop neckline Unlined Regular fit Dri Fit technology Polyester,Pink Rise/Heather/White/White,tops
2,FOREVER 21,Buffalo Plaid Crop Top,36.9,Checkered print puff sleeve crop top Square neckline Slim fit Polyblend,Black/Ivory,tops
3,Boohoo,Woven Button Back Shell Top,32.9,Crepe top with ruffle trim Round neckline Regular fit Back button fastening Short ruffled sleeves Hem with frill Polyester blend,Red,tops
4,Mama.licious,Maternity Patricia Long Sleeve Mix Top,59.0,Solid tone sheer plumeti sleeve top Round neckline Unlined Regular fit Back teardrop button fastening Viscose blend,Black,tops


In [101]:
all_zalora = [zal_jumpsuits, zal_pants, zal_skirts, zal_dresses, zal_shorts, zal_jackets, zal_blazers, zal_jeans, zal_cardi, zal_hoodies, zal_tops]
for df in all_zalora:
    print(df.isnull().sum())

brand          0
title          0
price          0
description    0
colour         0
category       0
dtype: int64
brand          0
title          0
price          0
description    0
colour         0
category       0
dtype: int64
brand          0
title          0
price          0
description    0
colour         0
category       0
dtype: int64
brand          0
title          0
price          0
description    0
colour         0
category       0
dtype: int64
brand          0
title          0
price          0
description    0
colour         0
category       0
dtype: int64
brand          0
title          0
price          0
description    0
colour         0
category       0
dtype: int64
brand          0
title          0
price          0
description    0
colour         0
category       0
dtype: int64
brand          0
title          0
price          0
description    0
colour         0
category       0
dtype: int64
brand          0
title          0
price          0
description    0
colour      

In [102]:
#fill empty colour columns with the word 'missing'
zal_tops['colour'].fillna('missing', inplace=True)

In [103]:
zalora=pd.concat(all_zalora)

In [104]:
zalora = zalora.reset_index(drop=True)

In [105]:
zalora.shape

(50467, 6)

In [106]:
zalora.to_csv("zalora.csv")

In [107]:
zalora.tail()

,brand,title,price,description,colour,category
50462,Dorothy Perkins,Navy Photographic Floral Top,69.9,Flowy top with floral print Round neckline Regular fit Slip on style Short tabbed sleeves Polyester,Navy Blue,tops
50463,ZALIA BASICS,Stand Up Collar Flare Sleeves Blouse,24.9,Flowy top with spots print High neckline Regular fit Back zip fastening Long flute sleeves High low hem Polyester,Black Polka Dots,tops
50464,A-IN GIRLS,Fashion Round Neck Sweater,199.5,"The Latest Korean Fashion Hot Selection, Easy Match. Knitting Thread Temperament Round Neck Stripe Design Product Production Varies From Supplier To Supplier",blue,tops
50465,Glamorous,Exaggerated Shoulder Long Sleeve Jumper,46.9,Solid tone puff sleeve jumper Unlined Round neckline Regular fit Viscose blend,Black,tops
50466,Calli,Lula Button Up Top,42.9,Monochrome sheer lace top Collared neckline Unlined Boxy fit Button fastening Cotton blend,Black Lace,tops


In [108]:
zalora.isnull().sum()

brand          0
title          0
price          0
description    0
colour         0
category       0
dtype: int64

## H&M

In [208]:
hm_tops = pd.read_csv("../datasets/hnm_tops.csv")
hm_mixed = pd.read_csv("../datasets/hnm_final (6).csv")
hm_mixed2 = pd.read_csv("../datasets/hnm_final (5).csv")
hm_dresses = pd.read_csv("../datasets/hnm_dresses.csv")

In [209]:
all_hm = [hm_tops, hm_mixed, hm_mixed2, hm_dresses]

In [210]:
for df in all_hm:
    print(df.columns)

Index(['web-scraper-order', 'web-scraper-start-url', 'link', 'link-href',
       'title', 'price', 'description', 'material', 'size', 'style', 'colour',
       'colour2'],
      dtype='object')
Index(['web-scraper-order', 'web-scraper-start-url', 'link', 'link-href',
       'title', 'price', 'description', 'material', 'size', 'style', 'colour',
       'colour2'],
      dtype='object')
Index(['web-scraper-order', 'web-scraper-start-url', 'link', 'link-href',
       'title', 'price', 'description', 'material', 'size', 'style', 'colour',
       'colour2'],
      dtype='object')
Index(['web-scraper-order', 'web-scraper-start-url', 'link', 'link-href',
       'title', 'price', 'description', 'material', 'size', 'style', 'colour',
       'colour2'],
      dtype='object')


In [211]:
hm_tops['category']='tops'
hm_dresses['category'] = 'dresses'

In [212]:
hm_mixed.head()

,web-scraper-order,web-scraper-start-url,link,link-href,title,price,description,material,size,style,colour,colour2
0,1573293361-11513,https://www2.hm.com/en_us/women/products/knit.html,Ribbed Mock-turtleneck Dress,https://www2.hm.com/en_us/productpage.0786347001.html,Ribbed Mock-turtleneck Dress,$24.99,Straight-cut dress in ribbed jersey with a mock turtleneck and long sleeves.,"Polyester 58%, Viscose 33%, Elastane 9%",Sleeve: Length: 60.5 cm (Size M) – cm = unitofmeasure,STYLE\n\t\t\t\t\n\t\t\t\t\tMock Turtleneck,Black,Black
1,1573291832-11114,https://www2.hm.com/en_us/women/products/lingerie.html,Super Push-up Lace Bra,https://www2.hm.com/en_us/productpage.0429313017.html,Super Push-up Lace Bra,$19.99,"Super push-up bra in lace with thickly padded, underwire cups to maximize bust and cleavage. Narrow, adjustable shoulder straps and a narrow section at back with two pairs of hook-and-eye fasteners.","Lace: Polyamide 88%, Elastane 12%",NaN,STYLE\n\t\t\t\t\n\t\t\t\t\tSuper Push-up Bra,Black/beige,Black/beige
2,1573291081-10917,https://www2.hm.com/en_us/women/products/lingerie.html,Strapless Balconette Bra,https://www2.hm.com/en_us/productpage.0699598004.html,Strapless Balconette Bra,$17.99,"Balconette bra in microfiber and lace with padded underwire cups for lifting and shaping. Narrow, detachable shoulder straps with multiple fastening options. Hook-and-eye fasteners at back.",Cup lining: Polyester 100%,NaN,STYLE\n\t\t\t\t\n\t\t\t\t\tStrapless Bra\n\t\t\t\t\tBalconette Bra\n\t\t\t\t\tUnderwire Bra\n\t\t\t\t\tPadded Bra,Light beige,Light beige
3,1573285039-9366,https://www2.hm.com/en_us/women/products/plus-size.html,H&M+ Slacks,https://www2.hm.com/en_us/productpage.0757552004.html,H&M+ Slacks,$24.99,"Ankle-length slacks in a stretch cotton blend with a regular waist. Concealed, elasticized waistband and zip fly with concealed hook-and-eye fasteners. Side pockets, mock welt back pockets, and tapered legs with creases.","Cotton 50%, Polyester 46%, Elastane 4%",Inner leg: Length: 71.9 cm (Size 16) – cm = unitofmeasure,STYLE\n\t\t\t\t\n\t\t\t\t\tTapered\n\t\t\t\t\tSlacks,Black,Black
4,1573288937-10393,https://www2.hm.com/en_us/women/products/sleepwear.html,Super Push-up Nightgown,https://www2.hm.com/en_us/productpage.0429322001.html,Super Push-up Nightgown,$29.99,"Nightgown in lace and mesh with an integral, super push-up bra with thickly padded, underwire cups to maximize bust and cleavage. Narrow, adjustable shoulder straps. Hook-and-eye fastening at back.","Lace: Polyamide 88%, Elastane 12%",NaN,STYLE\n\t\t\t\t\n\t\t\t\t\tUnderwire Bra\n\t\t\t\t\tSuper Push-up Bra\n\t\t\t\t\tPadded Bra,Black,Black


In [213]:
hm_mixed['web-scraper-start-url'].unique()

array(['https://www2.hm.com/en_us/women/products/knit.html',
       'https://www2.hm.com/en_us/women/products/lingerie.html',
       'https://www2.hm.com/en_us/women/products/plus-size.html',
       'https://www2.hm.com/en_us/women/products/sleepwear.html',
       'https://www2.hm.com/en_us/women/products/maternity-clothes.html',
       'https://www2.hm.com/en_us/women/products/workout-clothes.html'],
      dtype=object)

In [214]:
hm_mixed2['web-scraper-start-url'].unique()

array(['https://www2.hm.com/en_us/women/products/skirts.html',
       'https://www2.hm.com/en_us/women/products/pants.html',
       'https://www2.hm.com/en_us/women/products/jackets-coats.html',
       'https://www2.hm.com/en_us/women/products/basics.html',
       'https://www2.hm.com/en_us/women/products/cardigans-sweaters.html',
       'https://www2.hm.com/en_us/women/products/jumpsuits-rompers.html',
       'https://www2.hm.com/en_us/women/products/shorts.html',
       'https://www2.hm.com/en_us/women/products/jeans.html',
       'https://www2.hm.com/en_us/women/products/shirts-blouses.html',
       'https://www2.hm.com/en_us/women/products/hoodies-sweatshirts.html',
       'https://www2.hm.com/en_us/women/products/blazers-vests.html'],
      dtype=object)

In [215]:
hm_mixed['category']=0
for row in range(hm_mixed.shape[0]):
    if hm_mixed.loc[row,'web-scraper-start-url'] == 'https://www2.hm.com/en_us/women/products/knit.html':
        hm_mixed.loc[row,'category'] = 'knit'
    elif hm_mixed.loc[row,'web-scraper-start-url'] == 'https://www2.hm.com/en_us/women/products/lingerie.html':
        hm_mixed.loc[row,'category'] = 'lingerie'
    elif hm_mixed.loc[row,'web-scraper-start-url'] == 'https://www2.hm.com/en_us/women/products/plus-size.html':
        hm_mixed.loc[row,'category'] = 'plus size'
    elif hm_mixed.loc[row,'web-scraper-start-url'] == 'https://www2.hm.com/en_us/women/products/sleepwear.html':
        hm_mixed.loc[row,'category'] = 'loungewear'
    elif hm_mixed.loc[row,'web-scraper-start-url'] == 'https://www2.hm.com/en_us/women/products/maternity-clothes.html':
        hm_mixed.loc[row,'category'] = 'maternity'
    elif hm_mixed.loc[row,'web-scraper-start-url'] == 'https://www2.hm.com/en_us/women/products/workout-clothes.html':
        hm_mixed.loc[row,'category'] = 'activewear'

In [216]:
hm_mixed['category'].unique()

array(['knit', 'lingerie', 'plus size', 'loungewear', 'maternity',
       'activewear'], dtype=object)

In [217]:
hm_mixed2['category']=0
for row in range(hm_mixed2.shape[0]):
    if hm_mixed2.loc[row,'web-scraper-start-url'] == 'https://www2.hm.com/en_us/women/products/skirts.html':
        hm_mixed2.loc[row,'category'] = 'skirts'
    elif hm_mixed2.loc[row,'web-scraper-start-url'] == 'https://www2.hm.com/en_us/women/products/pants.html':
        hm_mixed2.loc[row,'category'] = 'pants'
    elif hm_mixed2.loc[row,'web-scraper-start-url'] == 'https://www2.hm.com/en_us/women/products/jackets-coats.html':
        hm_mixed2.loc[row,'category'] = 'outerwear'
    elif hm_mixed2.loc[row,'web-scraper-start-url'] == 'https://www2.hm.com/en_us/women/products/basics.html':
        hm_mixed2.loc[row,'category'] = 'basics'
    elif hm_mixed2.loc[row,'web-scraper-start-url'] == 'https://www2.hm.com/en_us/women/products/cardigans-sweaters.html':
        hm_mixed2.loc[row,'category'] = 'outerwear'
    elif hm_mixed2.loc[row,'web-scraper-start-url'] == 'https://www2.hm.com/en_us/women/products/jumpsuits-rompers.html':
        hm_mixed2.loc[row,'category'] = 'outerwear'
    elif hm_mixed2.loc[row,'web-scraper-start-url'] == 'https://www2.hm.com/en_us/women/products/shorts.html':
        hm_mixed2.loc[row,'category'] = 'shorts'
    elif hm_mixed2.loc[row,'web-scraper-start-url'] == 'https://www2.hm.com/en_us/women/products/jeans.html':
        hm_mixed2.loc[row,'category'] = 'jeans'
    elif hm_mixed2.loc[row,'web-scraper-start-url'] == 'https://www2.hm.com/en_us/women/products/shirts-blouses.html':
        hm_mixed2.loc[row,'category'] = 'tops'
    elif hm_mixed2.loc[row,'web-scraper-start-url'] == 'https://www2.hm.com/en_us/women/products/hoodies-sweatshirts.html':
        hm_mixed2.loc[row,'category'] = 'outerwear'
    elif hm_mixed2.loc[row,'web-scraper-start-url'] == 'https://www2.hm.com/en_us/women/products/blazers-vests.html':
        hm_mixed2.loc[row,'category'] = 'outerwear'

In [218]:
hm_mixed2['category'].unique()

array(['skirts', 'pants', 'outerwear', 'basics', 'shorts', 'jeans',
       'tops'], dtype=object)

In [219]:
hm_mixed2.groupby('category').count()

,web-scraper-order,web-scraper-start-url,link,link-href,title,price,description,material,size,style,colour,colour2
category,,,,,,,,,,,,
basics,578,578,578,578,578,578,578,577,570,475,578,578
jeans,240,240,240,240,240,240,240,240,240,100,239,240
outerwear,1275,1275,1275,1275,1274,1274,1274,1274,1258,954,1272,1274
pants,424,424,424,424,424,424,424,423,411,324,422,424
shorts,237,237,237,237,237,237,237,237,226,64,234,237
skirts,316,316,316,316,316,316,316,315,315,135,316,316
tops,526,526,526,526,526,526,526,526,520,469,525,526


In [220]:
hm_mixed2['description']=hm_mixed2['description'].str.lower()
hm_mixed2['title']=hm_mixed2['title'].str.lower()
hm_mixed['description']=hm_mixed['description'].str.lower()
hm_mixed['title']=hm_mixed['title'].str.lower()

In [221]:
for row in range(hm_mixed2.shape[0]):
    if hm_mixed2.loc[row,'category'] == 'basics':
        if 'top' in hm_mixed2['title'][row]:
            hm_mixed2.loc[row,'category'] = 'tops'
        elif 'blazers' in hm_mixed2['title'][row]:
            hm_mixed2.loc[row,'category'] = 'blazers'
        elif 'cardigans' in hm_mixed2['title'][row]:
            hm_mixed2.loc[row,'category'] = 'cardigans'
        elif 'hoodies' in hm_mixed2['title'][row]:
            hm_mixed2.loc[row,'category'] = 'hoodies'
        elif 'jackets' in hm_mixed2['title'][row]:
            hm_mixed2.loc[row,'category'] = 'jackets'
        elif 'jeans' in hm_mixed2['title'][row]:
            hm_mixed2.loc[row,'category'] = 'jeans'
        elif 'jumpers' in hm_mixed2['title'][row]:
            hm_mixed2.loc[row,'category'] = 'jumpers'
        elif 'pants' in hm_mixed2['title'][row]:
            hm_mixed2.loc[row,'category'] = 'pants'
        elif 'shorts' in hm_mixed2['title'][row]:
            hm_mixed2.loc[row,'category'] = 'shorts'
        elif 'skirts' in hm_mixed2['title'][row]:
            hm_mixed2.loc[row,'category'] = 'skirts'
        elif 'skirts' in hm_mixed2['title'][row]:
            hm_mixed2.loc[row,'category'] = 'skirts'
        elif 't-shirt' in hm_mixed2['title'][row]:
            hm_mixed2.loc[row,'category'] = 'tops'
        elif 'dress' in hm_mixed2['title'][row]:
            hm_mixed2.loc[row,'category'] = 'dress'
        elif 'sweatshirt' in hm_mixed2['title'][row]:
            hm_mixed2.loc[row,'category'] = 'outerwear'
        elif 'jacket' in hm_mixed2['title'][row]:
            hm_mixed2.loc[row,'category'] = 'outerwear'
        elif 'cardigan' in hm_mixed2['title'][row]:
            hm_mixed2.loc[row,'category'] = 'outerwear'
        elif 'sweater' in hm_mixed2['title'][row]:
            hm_mixed2.loc[row,'category'] = 'outerwear'
        elif 'joggers' in hm_mixed2['title'][row]:
            hm_mixed2.loc[row,'category'] = 'bottoms'
        elif 'leggings' in hm_mixed2['title'][row]:
            hm_mixed2.loc[row,'category'] = 'bottoms'
        elif 'shirt' in hm_mixed2['title'][row]:
            hm_mixed2.loc[row,'category'] = 'tops'
        elif 'bodysuit' in hm_mixed2['title'][row]:
            hm_mixed2.loc[row,'category'] = 'tops'
        elif 'brief' in hm_mixed2['title'][row]:
            hm_mixed2.loc[row,'category'] = 'lingerie'
        elif 'tights' in hm_mixed2['title'][row]:
            hm_mixed2.loc[row,'category'] = 'bottoms'

In [222]:
for row in range(hm_mixed.shape[0]):
    if hm_mixed.loc[row,'category'] == 'maternity':
        if 'top' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'tops'
        elif 'blazers' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'cardigans' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'hoodies' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'jackets' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'jeans' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'jeans'
        elif 'jumpers' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'pants' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'pants'
        elif 'shorts' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'shorts'
        elif 'skirts' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'skirts'
        elif 'skirt' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'skirts'
        elif 't-shirt' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'tops'
        elif 'dress' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'dress'
        elif 'sweatshirt' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'jacket' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'cardigan' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'sweater' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'joggers' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'bottoms'
        elif 'leggings' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'bottoms'
        elif 'shirt' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'tops'
        elif 'bodysuit' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'tops'
        elif 'brief' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'lingerie'
        elif 'tights' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'bottoms'
        elif 'bra' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'lingerie'
        elif 'coat' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'blouse' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'tops'
        elif 'parka' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'vest' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'jumpsuit' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'jumpsuit'
        elif 'pajama' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'loungewear'


In [223]:
for row in range(hm_mixed.shape[0]):
    if hm_mixed.loc[row,'category'] == 'plus size':
        if 'top' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'tops'
        elif 'blazers' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'blazers'
        elif 'cardigans' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'cardigans'
        elif 'hoodies' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'hoodies'
        elif 'jackets' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'jackets'
        elif 'jeans' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'jeans'
        elif 'jumpers' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'jumpers'
        elif 'pants' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'pants'
        elif 'shorts' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'shorts'
        elif 'skirts' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'skirts'
        elif 'skirt' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'skirts'
        elif 't-shirt' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'tops'
        elif 'dress' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'dress'
        elif 'sweatshirt' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'jacket' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'cardigan' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'sweater' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'joggers' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'bottoms'
        elif 'leggings' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'bottoms'
        elif 'shirt' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'tops'
        elif 'bodysuit' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'tops'
        elif 'brief' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'lingerie'
        elif 'tights' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'bottoms'
        elif 'bra' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'lingerie'
        elif 'coat' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'blouse' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'tops'
        elif 'parka' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'vest' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'jumpsuit' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'jumpsuit'
        elif 'pajama' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'loungewear'


In [224]:
for row in range(hm_mixed.shape[0]):
    if hm_mixed.loc[row,'category'] == 'activewear':
        if 'top' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'tops'
        elif 'blazers' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'blazers'
        elif 'cardigans' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'cardigans'
        elif 'hoodies' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'hoodies'
        elif 'jackets' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'jackets'
        elif 'jeans' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'jeans'
        elif 'jumpers' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'jumpers'
        elif 'pants' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'pants'
        elif 'shorts' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'shorts'
        elif 'skirts' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'skirts'
        elif 'skirt' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'skirts'
        elif 't-shirt' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'tops'
        elif 'dress' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'dress'
        elif 'sweatshirt' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'jacket' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'cardigan' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'sweater' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'joggers' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'bottoms'
        elif 'leggings' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'bottoms'
        elif 'shirt' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'tops'
        elif 'bodysuit' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'tops'
        elif 'brief' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'lingerie'
        elif 'tights' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'bottoms'
        elif 'bra' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'lingerie'
        elif 'coat' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'blouse' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'tops'
        elif 'parka' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'vest' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'jumpsuit' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'jumpsuit'
        elif 'pajama' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'loungewear'

In [225]:
for row in range(hm_mixed.shape[0]):
    if hm_mixed.loc[row,'category'] == 'knit':
        if 'top' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'tops'
        elif 'blazers' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'blazers'
        elif 'cardigans' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'cardigans'
        elif 'hoodies' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'hoodies'
        elif 'jackets' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'jackets'
        elif 'jeans' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'jeans'
        elif 'jumpers' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'jumpers'
        elif 'pants' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'pants'
        elif 'shorts' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'shorts'
        elif 'skirts' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'skirts'
        elif 'skirt' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'skirts'
        elif 't-shirt' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'tops'
        elif 'dress' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'dress'
        elif 'sweatshirt' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'jacket' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'cardigan' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'sweater' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'joggers' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'bottoms'
        elif 'leggings' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'bottoms'
        elif 'shirt' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'tops'
        elif 'bodysuit' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'tops'
        elif 'brief' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'lingerie'
        elif 'tights' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'bottoms'
        elif 'bra' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'lingerie'
        elif 'coat' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'blouse' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'tops'
        elif 'parka' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'vest' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'jumpsuit' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'jumpsuit'
        elif 'pajama' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'loungewear'
        elif 'poncho' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'
        elif 'turtleneck' in hm_mixed['title'][row]:
            hm_mixed.loc[row,'category'] = 'outerwear'

In [226]:
hm_mixed2.groupby('category').count().sort_values("link",ascending=False)

,web-scraper-order,web-scraper-start-url,link,link-href,title,price,description,material,size,style,colour,colour2
category,,,,,,,,,,,,
outerwear,1394,1394,1394,1394,1393,1393,1393,1393,1377,1054,1391,1393
tops,838,838,838,838,838,838,838,837,829,734,837,838
pants,442,442,442,442,442,442,442,441,429,342,440,442
skirts,316,316,316,316,316,316,316,315,315,135,316,316
jeans,240,240,240,240,240,240,240,240,240,100,239,240
shorts,237,237,237,237,237,237,237,237,226,64,234,237
dress,78,78,78,78,78,78,78,78,78,64,78,78
bottoms,41,41,41,41,41,41,41,41,37,18,41,41
lingerie,8,8,8,8,8,8,8,8,7,8,8,8


In [227]:
# tofix = hm_mixed[hm_mixed.loc[:,'category'] == 'knit']
# tofix

In [228]:
features = ['title','price','description','material','colour','colour2','category']

In [229]:
hm_final = pd.concat([hm_mixed,hm_mixed2,hm_tops,hm_dresses])

In [230]:
hm_final = hm_final[features]

In [231]:
hm_final.head()

,title,price,description,material,colour,colour2,category
0,ribbed mock-turtleneck dress,$24.99,straight-cut dress in ribbed jersey with a mock turtleneck and long sleeves.,"Polyester 58%, Viscose 33%, Elastane 9%",Black,Black,dress
1,super push-up lace bra,$19.99,"super push-up bra in lace with thickly padded, underwire cups to maximize bust and cleavage. narrow, adjustable shoulder straps and a narrow section at back with two pairs of hook-and-eye fasteners.","Lace: Polyamide 88%, Elastane 12%",Black/beige,Black/beige,lingerie
2,strapless balconette bra,$17.99,"balconette bra in microfiber and lace with padded underwire cups for lifting and shaping. narrow, detachable shoulder straps with multiple fastening options. hook-and-eye fasteners at back.",Cup lining: Polyester 100%,Light beige,Light beige,lingerie
3,h&m+ slacks,$24.99,"ankle-length slacks in a stretch cotton blend with a regular waist. concealed, elasticized waistband and zip fly with concealed hook-and-eye fasteners. side pockets, mock welt back pockets, and tapered legs with creases.","Cotton 50%, Polyester 46%, Elastane 4%",Black,Black,plus size
4,super push-up nightgown,$29.99,"nightgown in lace and mesh with an integral, super push-up bra with thickly padded, underwire cups to maximize bust and cleavage. narrow, adjustable shoulder straps. hook-and-eye fastening at back.","Lace: Polyamide 88%, Elastane 12%",Black,Black,loungewear


In [232]:
hm_final.description = hm_final.description + " " + hm_final.material

In [233]:
hm_final.drop(columns=['material'],inplace=True)

In [234]:
hm_final= hm_final.reset_index(drop=True)
hm_final.head()

,title,price,description,colour,colour2,category
0,ribbed mock-turtleneck dress,$24.99,"straight-cut dress in ribbed jersey with a mock turtleneck and long sleeves. Polyester 58%, Viscose 33%, Elastane 9%",Black,Black,dress
1,super push-up lace bra,$19.99,"super push-up bra in lace with thickly padded, underwire cups to maximize bust and cleavage. narrow, adjustable shoulder straps and a narrow section at back with two pairs of hook-and-eye fasteners. Lace: Polyamide 88%, Elastane 12%",Black/beige,Black/beige,lingerie
2,strapless balconette bra,$17.99,"balconette bra in microfiber and lace with padded underwire cups for lifting and shaping. narrow, detachable shoulder straps with multiple fastening options. hook-and-eye fasteners at back. Cup lining: Polyester 100%",Light beige,Light beige,lingerie
3,h&m+ slacks,$24.99,"ankle-length slacks in a stretch cotton blend with a regular waist. concealed, elasticized waistband and zip fly with concealed hook-and-eye fasteners. side pockets, mock welt back pockets, and tapered legs with creases. Cotton 50%, Polyester 46%, Elastane 4%",Black,Black,plus size
4,super push-up nightgown,$29.99,"nightgown in lace and mesh with an integral, super push-up bra with thickly padded, underwire cups to maximize bust and cleavage. narrow, adjustable shoulder straps. hook-and-eye fastening at back. Lace: Polyamide 88%, Elastane 12%",Black,Black,loungewear


In [235]:
for row in range(hm_final.shape[0]):
    if hm_final.loc[row,'colour'] != hm_final.loc[row,'colour2']:
        hm_final.loc[row,'colour'] = str(hm_final.loc[row,'colour']) + " " +str(hm_final.loc[row,'colour2'])
        
    

In [236]:
hm_final['colour'] = hm_final['colour'].str.replace('/',' ')
hm_final.drop(columns=['colour2'],inplace=True)

In [237]:

hm_final.head()

,title,price,description,colour,category
0,ribbed mock-turtleneck dress,$24.99,"straight-cut dress in ribbed jersey with a mock turtleneck and long sleeves. Polyester 58%, Viscose 33%, Elastane 9%",Black,dress
1,super push-up lace bra,$19.99,"super push-up bra in lace with thickly padded, underwire cups to maximize bust and cleavage. narrow, adjustable shoulder straps and a narrow section at back with two pairs of hook-and-eye fasteners. Lace: Polyamide 88%, Elastane 12%",Black beige,lingerie
2,strapless balconette bra,$17.99,"balconette bra in microfiber and lace with padded underwire cups for lifting and shaping. narrow, detachable shoulder straps with multiple fastening options. hook-and-eye fasteners at back. Cup lining: Polyester 100%",Light beige,lingerie
3,h&m+ slacks,$24.99,"ankle-length slacks in a stretch cotton blend with a regular waist. concealed, elasticized waistband and zip fly with concealed hook-and-eye fasteners. side pockets, mock welt back pockets, and tapered legs with creases. Cotton 50%, Polyester 46%, Elastane 4%",Black,plus size
4,super push-up nightgown,$29.99,"nightgown in lace and mesh with an integral, super push-up bra with thickly padded, underwire cups to maximize bust and cleavage. narrow, adjustable shoulder straps. hook-and-eye fastening at back. Lace: Polyamide 88%, Elastane 12%",Black,loungewear


In [238]:
hm_final['title'] = hm_final['title'].str.lower()
hm_final['description'] = hm_final['description'].str.lower()
hm_final['colour'] = hm_final['colour'].str.lower()

In [239]:
hm_final['title'] = hm_final['title'].str.replace('-',' ')
hm_final['description'] = hm_final['description'].str.replace('-',' ')
hm_final['price'] = hm_final['price'].str.replace('$','')

In [240]:
hm_final.isnull().sum()

title          1
price          1
description    5
colour         0
category       0
dtype: int64

In [241]:
hm_final = hm_final.dropna(axis=0, subset=['price'])

In [242]:
hm_final['description'].fillna(hm_final['title'],inplace=True)

In [243]:
hm_final.to_csv("hnm.csv")

## Zara

In [119]:
zara = pd.read_csv("../datasets/zara.csv")
#zara2 = pd.read_csv("../datasets/zara_sg.csv")

In [120]:
zara.head()

,web-scraper-order,web-scraper-start-url,link,link-href,title,price,colour,description,material
0,1573331974-12693,https://www.zara.com/sg/en/woman-tshirts-l1362.html?v1=1281620,TOP WITH CONTRASTING COLLAR,https://www.zara.com/sg/en/top-with-contrasting-collar-p05584463.html,TOP WITH CONTRASTING COLLAR Details,S$ 45.90,Black,Long sleeve top with a matching high neck and gathers on the shoulders. \n\nHEIGHT OF MODEL: 177 cm. / 69.6″,NaN
1,1573336501-14065,https://www.zara.com/sg/en/woman-shirts-l1217.html?v1=1281626,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1573332833-12940,https://www.zara.com/sg/en/woman-skirts-l1299.html?v1=1281622,FAUX LEATHER SKIRT,https://www.zara.com/sg/en/faux-leather-skirt-p02969243.html,FAUX LEATHER SKIRT Details,S$ 69.90,Black,Pleated mini skirt with an elastic waist and interior lining.\n\nHEIGHT OF MODEL: 177 CM / 5′ 9″,NaN
3,1573335223-13640,https://www.zara.com/sg/en/woman-knitwear-l1152.html?v1=1281621,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1573337664-14379,https://www.zara.com/sg/en/woman-shirts-l1217.html?v1=1281626,PRINTED DRAPED TOP,https://www.zara.com/sg/en/printed-draped-top-p09006152.html,PRINTED DRAPED TOP Details,S$ 59.90,Black,"V-neck crop top with long sleeves, ruched detail on the front and elastic on the back.\n\nHEIGHT OF MODEL: 177 cm. / 5′ 9″",NaN


In [121]:
zara['web-scraper-start-url'].unique()

array(['https://www.zara.com/sg/en/woman-tshirts-l1362.html?v1=1281620',
       'https://www.zara.com/sg/en/woman-shirts-l1217.html?v1=1281626',
       'https://www.zara.com/sg/en/woman-skirts-l1299.html?v1=1281622',
       'https://www.zara.com/sg/en/woman-knitwear-l1152.html?v1=1281621',
       'https://www.zara.com/sg/en/woman-dresses-l1066.html?v1=1281625',
       'https://www.zara.com/sg/en/woman-blazers-l1055.html?v1=1281650',
       'https://www.zara.com/sg/en/woman-trousers-l1335.html?v1=1281627',
       'https://www.zara.com/sg/en/woman-jeans-l1119.html?v1=1281624',
       'https://www.zara.com/sg/en/woman-outerwear-l1184.html?v1=1281618',
       'https://www.zara.com/sg/en/woman-suits-l1437.html?v1=1281679',
       'https://www.zara.com/sg/en/woman-jackets-l1114.html?v1=1281623',
       'https://www.zara.com/sg/en/woman-sweatshirts-l1320.html?v1=1281548',
       'https://www.zara.com/sg/en/woman-basics-l1050.html?v1=1282128'],
      dtype=object)

In [122]:
zara['category']=0
for row in range(zara.shape[0]):
    if zara.loc[row,'web-scraper-start-url'] == 'https://www.zara.com/sg/en/woman-tshirts-l1362.html?v1=1281620':
        zara.loc[row,'category'] = 'tops'
    elif zara.loc[row,'web-scraper-start-url'] == 'https://www.zara.com/sg/en/woman-shirts-l1217.html?v1=1281626':
        zara.loc[row,'category'] = 'tops'
    elif zara.loc[row,'web-scraper-start-url'] == 'https://www.zara.com/sg/en/woman-skirts-l1299.html?v1=1281622':
        zara.loc[row,'category'] = 'skirts'
    elif zara.loc[row,'web-scraper-start-url'] == 'https://www.zara.com/sg/en/woman-knitwear-l1152.html?v1=1281621':
        zara.loc[row,'category'] = 'outerwear'
    elif zara.loc[row,'web-scraper-start-url'] == 'https://www.zara.com/sg/en/woman-dresses-l1066.html?v1=1281625':
        zara.loc[row,'category'] = 'dresses'
    elif zara.loc[row,'web-scraper-start-url'] == 'https://www.zara.com/sg/en/woman-trousers-l1335.html?v1=1281627':
        zara.loc[row,'category'] = 'bottoms'
    elif zara.loc[row,'web-scraper-start-url'] == 'https://www.zara.com/sg/en/woman-jeans-l1119.html?v1=1281624':
        zara.loc[row,'category'] = 'jeans'
    elif zara.loc[row,'web-scraper-start-url'] == 'https://www.zara.com/sg/en/woman-outerwear-l1184.html?v1=1281618':
        zara.loc[row,'category'] = 'outerwear'
    elif zara.loc[row,'web-scraper-start-url'] == 'https://www.zara.com/sg/en/woman-suits-l1437.html?v1=1281679':
        zara.loc[row,'category'] = 'outerwear'
    elif zara.loc[row,'web-scraper-start-url'] == 'https://www.zara.com/sg/en/woman-jackets-l1114.html?v1=1281623':
        zara.loc[row,'category'] = 'outerwear'
    elif zara.loc[row,'web-scraper-start-url'] == 'https://www.zara.com/sg/en/woman-sweatshirts-l1320.html?v1=1281548':
        zara.loc[row,'category'] = 'outerwear'
    elif zara.loc[row,'web-scraper-start-url'] == 'https://www.zara.com/sg/en/woman-blazers-l1055.html?v1=1281650':
        zara.loc[row,'category'] = 'outerwear'
    elif zara.loc[row,'web-scraper-start-url'] == 'https://www.zara.com/sg/en/woman-basics-l1050.html?v1=1282128':
        zara.drop(row, inplace=True)
    

In [123]:
zara.shape

(3883, 10)

In [124]:
zara.material.unique()

array([nan])

In [125]:
zara.isnull().sum()

web-scraper-order           0
web-scraper-start-url       0
link                      373
link-href                 369
title                     370
price                     386
colour                    374
description               380
material                 3883
category                    0
dtype: int64

In [127]:
zara.category.unique()

array(['tops', 'skirts', 'outerwear', 'dresses', 'bottoms', 'jeans'],
      dtype=object)

In [128]:
features = ['title','price','colour','description','category']
zara = zara[features]
    

In [129]:
zara['brand'] = 'zara'

In [130]:
zara.head()

,title,price,colour,description,category,brand
0,TOP WITH CONTRASTING COLLAR Details,S$ 45.90,Black,Long sleeve top with a matching high neck and gathers on the shoulders. \n\nHEIGHT OF MODEL: 177 cm. / 69.6″,tops,zara
1,NaN,NaN,NaN,NaN,tops,zara
2,FAUX LEATHER SKIRT Details,S$ 69.90,Black,Pleated mini skirt with an elastic waist and interior lining.\n\nHEIGHT OF MODEL: 177 CM / 5′ 9″,skirts,zara
3,NaN,NaN,NaN,NaN,outerwear,zara
4,PRINTED DRAPED TOP Details,S$ 59.90,Black,"V-neck crop top with long sleeves, ruched detail on the front and elastic on the back.\n\nHEIGHT OF MODEL: 177 cm. / 5′ 9″",tops,zara


In [131]:

zara['price'] = zara['price'].astype(str)
zara['price']=[x[3:] for x in zara.price]
zara['price'] = pd.to_numeric(zara['price'], errors='coerce').fillna(0)

In [132]:
zara = zara[zara['price']>0].reset_index(drop=True)

In [133]:
zara.isnull().sum()

title           1
price           0
colour          5
description    11
category        0
brand           0
dtype: int64

In [134]:
zara['colour'].fillna('missing', inplace=True)

In [135]:
zara['description'].fillna(zara['title'],inplace=True)

In [136]:
zara[zara.isnull().any(axis=1)]

,title,price,colour,description,category,brand
1460,NaN,45.9,missing,NaN,outerwear,zara


In [137]:
zara.drop(1460,inplace=True)

In [138]:
zara = zara.reset_index(drop=True)

In [139]:
zara['description'] = zara['description'].astype(str)

for row in range(zara.shape[0]):
        zara.loc[row, 'description'] = zara.loc[row,'description'].split("\n\n")[0]

In [140]:
zara['title'] = zara['title'].str.lower()
zara['description'] = zara['description'].str.lower()
zara['colour'] = zara['colour'].str.lower()

In [268]:
zara['title'] = zara['title'].str.replace('-',' ')
zara['description'] = zara['description'].str.replace('-',' ')


In [269]:
zara.isnull().sum()

title          0
price          0
colour         0
description    0
category       0
brand          0
dtype: int64

In [270]:
zara.to_csv("zara.csv")

## Runway Bandits

In [279]:
rb = pd.read_csv('../datasets/runwaybandits.csv')

In [280]:
rb.shape

(1447, 7)

In [281]:
rb.head()

,Unnamed: 0,url,brand,title,price,description,colour
0,0,https://www.runwaybandits.com/product/clothes/dresses/resa-dress-sky,Runway Bandits,RESA DRESS,SGD $49.00,"Faux wrap designs are a highly popular feature in both blouses and dresses, and it’s not hard to see why. The",COLOUR:\nSKY
1,1,https://www.runwaybandits.com/product/clothes/dresses/ninya-dress-blush,Runway Bandits,NINYA DRESS,SGD $49.00,"Long, flowy, and feminine, the",COLOUR:\nBLUSH
2,2,https://www.runwaybandits.com/product/clothes/dresses/resa-dress-dark-pine-green,Runway Bandits,RESA DRESS,SGD $49.00,"Faux wrap designs are a highly popular feature in both blouses and dresses, and it’s not hard to see why. The",COLOUR:\nDARK PINE GREEN
3,3,https://www.runwaybandits.com/product/clothes/dresses/jireh-dress-navy,Runway Bandits,JIREH DRESS,SGD $42.00,Think of the,COLOUR:\nNAVY
4,4,https://www.runwaybandits.com/product/clothes/tops/gilbet-shirt-emerald,Runway Bandits,GILBET SHIRT,SGD $33.00,"Everybody needs a basic long-sleeved button-down shirt in their wardrobe and if you don’t already have one, why not consider the",COLOUR:\nEMERALD


In [282]:
rb.drop(columns=['Unnamed: 0'], inplace=True)

In [283]:
rb['price'] = rb['price'].str.replace("SGD $","")

In [284]:
rb.head()

,url,brand,title,price,description,colour
0,https://www.runwaybandits.com/product/clothes/dresses/resa-dress-sky,Runway Bandits,RESA DRESS,SGD $49.00,"Faux wrap designs are a highly popular feature in both blouses and dresses, and it’s not hard to see why. The",COLOUR:\nSKY
1,https://www.runwaybandits.com/product/clothes/dresses/ninya-dress-blush,Runway Bandits,NINYA DRESS,SGD $49.00,"Long, flowy, and feminine, the",COLOUR:\nBLUSH
2,https://www.runwaybandits.com/product/clothes/dresses/resa-dress-dark-pine-green,Runway Bandits,RESA DRESS,SGD $49.00,"Faux wrap designs are a highly popular feature in both blouses and dresses, and it’s not hard to see why. The",COLOUR:\nDARK PINE GREEN
3,https://www.runwaybandits.com/product/clothes/dresses/jireh-dress-navy,Runway Bandits,JIREH DRESS,SGD $42.00,Think of the,COLOUR:\nNAVY
4,https://www.runwaybandits.com/product/clothes/tops/gilbet-shirt-emerald,Runway Bandits,GILBET SHIRT,SGD $33.00,"Everybody needs a basic long-sleeved button-down shirt in their wardrobe and if you don’t already have one, why not consider the",COLOUR:\nEMERALD


In [285]:
rb['price'] = rb['price'].astype(str)
rb['price']=[x[5:] for x in rb.price]

In [286]:
rb['colour'] = rb['colour'].astype(str)
rb['colour']=[x[8:] for x in rb.colour]

In [287]:
rb['colour'] = rb['colour'].str.lower()
rb['title'] = rb['title'].str.lower()
rb['description'] = rb['description'].str.lower()

In [288]:
rb.url.unique()

array(['https://www.runwaybandits.com/product/clothes/dresses/resa-dress-sky',
       'https://www.runwaybandits.com/product/clothes/dresses/ninya-dress-blush',
       'https://www.runwaybandits.com/product/clothes/dresses/resa-dress-dark-pine-green',
       ...,
       'https://www.runwaybandits.com/product/clothes/tops/hilby-cami-turqoise',
       'https://www.runwaybandits.com/product/clothes/pants/nala-pants-blue',
       'https://www.runwaybandits.com/product/clothes/tops/la-foce-cami-black'],
      dtype=object)

In [289]:
rb['category']=0
for row in range(rb.shape[0]):
    if 'dress' in rb.loc[row,'url']:
        rb.loc[row,'category'] = 'dress'
    elif 'outerwear' in rb.loc[row,'url']:
        rb.loc[row,'category'] = 'outerwear'
    elif 'pants' in rb.loc[row,'url']:
        rb.loc[row,'category'] = 'bottoms'
    elif 'shorts' in rb.loc[row,'url']:
        rb.loc[row,'category'] = 'shorts'
    elif 'skirts' in rb.loc[row,'url']:
        rb.loc[row,'category'] = 'skirts'
    elif 'tops' in rb.loc[row,'url']:
        rb.loc[row,'category'] = 'tops' 
    elif 'jumpsuit' in rb.loc[row,'url']:
        rb.loc[row,'category'] = 'jumpsuits' 
    elif 'romper' in rb.loc[row,'url']:
        rb.loc[row,'category'] = 'jumpsuits' 
    elif 'one-piece' in rb.loc[row,'url']:
        rb.loc[row,'category'] = 'jumpsuits' 


In [290]:
features = ['title','price','brand','description','colour','category']
rb = rb[features]

In [291]:
rb.head()

,title,price,brand,description,colour,category
0,resa dress,49.00,Runway Bandits,"faux wrap designs are a highly popular feature in both blouses and dresses, and it’s not hard to see why. the",sky,dress
1,ninya dress,49.00,Runway Bandits,"long, flowy, and feminine, the",blush,dress
2,resa dress,49.00,Runway Bandits,"faux wrap designs are a highly popular feature in both blouses and dresses, and it’s not hard to see why. the",dark pine green,dress
3,jireh dress,42.00,Runway Bandits,think of the,navy,dress
4,gilbet shirt,33.00,Runway Bandits,"everybody needs a basic long-sleeved button-down shirt in their wardrobe and if you don’t already have one, why not consider the",emerald,tops


In [292]:
rb.isnull().sum()

title          0
price          0
brand          0
description    0
colour         0
category       0
dtype: int64

In [293]:
rb.to_csv("rb.csv")

## Our Second Nature

In [312]:
osn = pd.read_csv("../datasets/oursecondnature.csv")

In [319]:
osn.head()

,url,brand,title,price,description,colour,material
0,https://oursecondnature.com/home/323-sn323-SN32346XS.html#/size-xs/colours-white_print,Our Second Nature,Fragments Cotton Midi Dress,109.00,Made for buffets. Be comfortable all day long in our Cotton Midi Dress. This piece features a V-neckline and comes with two side pockets as well as a concealed side zipper for ease of wearing. Dress is fully lined and true to size. Print placement is fixed for each piece.,"['White Print', 'Green Print']",Composition 100% Cotton
1,https://oursecondnature.com/home/323-sn323.html#/size-xs/colours-green_print,Our Second Nature,Fragments Cotton Midi Dress,109.00,Made for buffets. Be comfortable all day long in our Cotton Midi Dress. This piece features a V-neckline and comes with two side pockets as well as a concealed side zipper for ease of wearing. Dress is fully lined and true to size. Print placement is fixed for each piece.,"['White Print', 'Green Print']",Composition 100% Cotton
2,https://oursecondnature.com/home/323-sn323-SN32346XS.html#/size-xs/colours-white_print,Our Second Nature,Fragments Cotton Midi Dress,109.00,Made for buffets. Be comfortable all day long in our Cotton Midi Dress. This piece features a V-neckline and comes with two side pockets as well as a concealed side zipper for ease of wearing. Dress is fully lined and true to size. Print placement is fixed for each piece.,"['White Print', 'Green Print']",Composition 100% Cotton
3,https://oursecondnature.com/home/323-sn323.html#/size-xs/colours-green_print,Our Second Nature,Fragments Cotton Midi Dress,109.00,Made for buffets. Be comfortable all day long in our Cotton Midi Dress. This piece features a V-neckline and comes with two side pockets as well as a concealed side zipper for ease of wearing. Dress is fully lined and true to size. Print placement is fixed for each piece.,"['White Print', 'Green Print']",Composition 100% Cotton
4,https://oursecondnature.com/home/303-sn303.html#/size-xs/colours-white,Our Second Nature,Gathered Shoulder Crop Top,59.00,"Our classic v-neck crop top in its signature boxy silhouette, now with added detailing at the shoulders. Pair it your favourite high-waisted bottoms and you're all set.","['', 'Yellow', 'Blue']",Composition 100% Cotton


In [314]:
osn.drop(columns=['Unnamed: 0'], inplace=True)

In [315]:
osn['price'] = osn['price'].astype(str)
osn['price']=[x[4:] for x in osn.price]

In [318]:
osn['material'] = osn['material'].str.replace(":&nbsp;", " ")

In [321]:
def remove_duplicates(df):
    before_d = len(df)
    df.drop_duplicates(inplace=True)
    after_d = len(df)
    print(str(before_d-after_d) + " duplicates were removed!")


In [322]:
remove_duplicates(osn)

477 duplicates were removed!


In [324]:
osn.shape

(701, 7)

In [325]:
osn.head()

,url,brand,title,price,description,colour,material
0,https://oursecondnature.com/home/323-sn323-SN32346XS.html#/size-xs/colours-white_print,Our Second Nature,Fragments Cotton Midi Dress,109.00,Made for buffets. Be comfortable all day long in our Cotton Midi Dress. This piece features a V-neckline and comes with two side pockets as well as a concealed side zipper for ease of wearing. Dress is fully lined and true to size. Print placement is fixed for each piece.,"['White Print', 'Green Print']",Composition 100% Cotton
1,https://oursecondnature.com/home/323-sn323.html#/size-xs/colours-green_print,Our Second Nature,Fragments Cotton Midi Dress,109.00,Made for buffets. Be comfortable all day long in our Cotton Midi Dress. This piece features a V-neckline and comes with two side pockets as well as a concealed side zipper for ease of wearing. Dress is fully lined and true to size. Print placement is fixed for each piece.,"['White Print', 'Green Print']",Composition 100% Cotton
4,https://oursecondnature.com/home/303-sn303.html#/size-xs/colours-white,Our Second Nature,Gathered Shoulder Crop Top,59.00,"Our classic v-neck crop top in its signature boxy silhouette, now with added detailing at the shoulders. Pair it your favourite high-waisted bottoms and you're all set.","['', 'Yellow', 'Blue']",Composition 100% Cotton
5,https://oursecondnature.com/home/303-sn303.html#/colours-yellow/size-xs,Our Second Nature,Gathered Shoulder Crop Top,59.00,"Our classic v-neck crop top in its signature boxy silhouette, now with added detailing at the shoulders. Pair it your favourite high-waisted bottoms and you're all set.","['', 'Yellow', 'Blue']",Composition 100% Cotton
6,https://oursecondnature.com/home/303-sn303.html#/size-xs/colours-blue,Our Second Nature,Gathered Shoulder Crop Top,59.00,"Our classic v-neck crop top in its signature boxy silhouette, now with added detailing at the shoulders. Pair it your favourite high-waisted bottoms and you're all set.","['', 'Yellow', 'Blue']",Composition 100% Cotton


In [331]:
osn = osn.reset_index(drop=True)

In [339]:
osn['colour2'] = 0
for row in range(osn.shape[0]):
        osn.loc[row, 'colour2'] = osn.loc[row,'url'].split("/colours")[1]
        osn.loc[row, 'colour2'] = osn.loc[row,'colour2'].split("/")[0]       

In [342]:
osn.head()

,url,brand,title,price,description,colour,material,colour2
0,https://oursecondnature.com/home/323-sn323-SN32346XS.html#/size-xs/colours-white_print,Our Second Nature,Fragments Cotton Midi Dress,109.00,Made for buffets. Be comfortable all day long in our Cotton Midi Dress. This piece features a V-neckline and comes with two side pockets as well as a concealed side zipper for ease of wearing. Dress is fully lined and true to size. Print placement is fixed for each piece.,"['White Print', 'Green Print']",Composition 100% Cotton,white print
1,https://oursecondnature.com/home/323-sn323.html#/size-xs/colours-green_print,Our Second Nature,Fragments Cotton Midi Dress,109.00,Made for buffets. Be comfortable all day long in our Cotton Midi Dress. This piece features a V-neckline and comes with two side pockets as well as a concealed side zipper for ease of wearing. Dress is fully lined and true to size. Print placement is fixed for each piece.,"['White Print', 'Green Print']",Composition 100% Cotton,green print
2,https://oursecondnature.com/home/303-sn303.html#/size-xs/colours-white,Our Second Nature,Gathered Shoulder Crop Top,59.00,"Our classic v-neck crop top in its signature boxy silhouette, now with added detailing at the shoulders. Pair it your favourite high-waisted bottoms and you're all set.","['', 'Yellow', 'Blue']",Composition 100% Cotton,white
3,https://oursecondnature.com/home/303-sn303.html#/colours-yellow/size-xs,Our Second Nature,Gathered Shoulder Crop Top,59.00,"Our classic v-neck crop top in its signature boxy silhouette, now with added detailing at the shoulders. Pair it your favourite high-waisted bottoms and you're all set.","['', 'Yellow', 'Blue']",Composition 100% Cotton,yellow
4,https://oursecondnature.com/home/303-sn303.html#/size-xs/colours-blue,Our Second Nature,Gathered Shoulder Crop Top,59.00,"Our classic v-neck crop top in its signature boxy silhouette, now with added detailing at the shoulders. Pair it your favourite high-waisted bottoms and you're all set.","['', 'Yellow', 'Blue']",Composition 100% Cotton,blue


In [341]:
for row in range(osn.shape[0]):
        osn.loc[row, 'colour2'] = osn.loc[row,'colour2'].replace("-","")
        osn.loc[row, 'colour2'] = osn.loc[row,'colour2'].replace("_"," ")

In [343]:
osn.isnull().sum()

url            0
brand          0
title          0
price          0
description    0
colour         0
material       0
colour2        0
dtype: int64

In [345]:
features = ['brand','title','price','description','material','colour2']
osn = osn[features]

In [348]:
osn.rename(columns={'colour2':'colour'},inplace=True)
osn.head()

,brand,title,price,description,material,colour
0,Our Second Nature,Fragments Cotton Midi Dress,109.00,Made for buffets. Be comfortable all day long in our Cotton Midi Dress. This piece features a V-neckline and comes with two side pockets as well as a concealed side zipper for ease of wearing. Dress is fully lined and true to size. Print placement is fixed for each piece.,Composition 100% Cotton,white print
1,Our Second Nature,Fragments Cotton Midi Dress,109.00,Made for buffets. Be comfortable all day long in our Cotton Midi Dress. This piece features a V-neckline and comes with two side pockets as well as a concealed side zipper for ease of wearing. Dress is fully lined and true to size. Print placement is fixed for each piece.,Composition 100% Cotton,green print
2,Our Second Nature,Gathered Shoulder Crop Top,59.00,"Our classic v-neck crop top in its signature boxy silhouette, now with added detailing at the shoulders. Pair it your favourite high-waisted bottoms and you're all set.",Composition 100% Cotton,white
3,Our Second Nature,Gathered Shoulder Crop Top,59.00,"Our classic v-neck crop top in its signature boxy silhouette, now with added detailing at the shoulders. Pair it your favourite high-waisted bottoms and you're all set.",Composition 100% Cotton,yellow
4,Our Second Nature,Gathered Shoulder Crop Top,59.00,"Our classic v-neck crop top in its signature boxy silhouette, now with added detailing at the shoulders. Pair it your favourite high-waisted bottoms and you're all set.",Composition 100% Cotton,blue


In [353]:
osn['category'] = 0
for row in range(osn.shape[0]):
    if "Top" in osn.loc[row,'title']:
        osn.loc[row,'category'] = "tops"
    elif "Dress" in osn.loc[row,'title']:
        osn.loc[row,'category'] = "dress"
    elif "Shorts" in osn.loc[row,'title']:
        osn.loc[row,'category'] = "shorts"
    elif "Skirt" in osn.loc[row,'title']:
        osn.loc[row,'category'] = "skirts"
    elif "Shirt" in osn.loc[row,'title']:
        osn.loc[row,'category'] = "tops"
    elif "Jumpsuit" in osn.loc[row,'title']:
        osn.loc[row,'category'] = "jumpsuits"
    elif "Pants" in osn.loc[row,'title']:
        osn.loc[row,'category'] = "bottoms"
    elif "Blouse" in osn.loc[row,'title']:
        osn.loc[row,'category'] = "tops"
    elif "Culottes" in osn.loc[row,'title']:
        osn.loc[row,'category'] = "bottoms"
    elif "Tee" in osn.loc[row,'title']:
        osn.loc[row,'category'] = "tops"
    elif "Kimono" in osn.loc[row,'title']:
        osn.loc[row,'category'] = "outerwear"
    elif "Kaftan" in osn.loc[row,'title']:
        osn.loc[row,'category'] = "outerwear"
    elif "Blazer" in osn.loc[row,'title']:
        osn.loc[row,'category'] = "outerwear"
    elif "Vest" in osn.loc[row,'title']:
        osn.loc[row,'category'] = "outerwear"
    elif "Cami" in osn.loc[row,'title']:
        osn.loc[row,'category'] = "tops"
        

/Users/lynn/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/lynn/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/lynn/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cw

In [355]:
for row in range(osn.shape[0]):
    if osn.loc[row,'category'] == 0:
        osn.drop(row,inplace=True)

/Users/lynn/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [356]:
osn = osn.reset_index(drop=True)

In [357]:
osn.to_csv("osn.csv")

In [359]:
osn.shape

(684, 7)

## Love Bonito

In [478]:
lb = pd.read_csv("../datasets/lb2.csv")

In [479]:
lb.shape

(50763, 8)

In [480]:
lb = lb.dropna(axis=0, subset=['title 1'])

In [481]:
lb.shape

(1893, 8)

In [482]:
lb = lb.reset_index(drop=True)

In [483]:
lb.head()

,Address,Status Code,Status,title 1,price 1,colour 1,material 1,Description 1 1
0,https://www.lovebonito.com/sg/meisel-shawl-collar-blazer.html,200,OK,Meisel Shawl Collar Blazer,S$ 51.90,NaN,<br>100% Polyester,"Suit up, ladies. Gathered sleeve detail. Open front. <br>Model Cindy is 178cm tall, a UK6/8, wears Size S.\r\n"
1,https://www.lovebonito.com/sg/blaise-midi-polka-dot-dress.html,200,OK,Blaise Midi Polka Dot Dress,S$ 51.90,NaN,<br>100% Polyester,"Flair for the dramatic. Bold polkadot print. Ruched waist. Puff sleeves.<br>Model MM is 176cm tall, a UK6/8, wears Size S.\r\n"
2,https://www.lovebonito.com/sg/marley-printed-eco-bag-in-floral-waltz.html,200,OK,Marley Printed Eco Bag in Floral Waltz,S$ 14.90,NaN,NaN,<b>Specially designed in-house print.</b> Responsibly chic. Twin handles.<br>
3,https://www.lovebonito.com/sg/jeryl-ribbon-tie-top.html,200,OK,Jeryl Ribbon Tie Top,S$ 36.90,NaN,<br>100% Polyester,"For your day-time thing. Ribbon tie straps. Button details. V-neckline. \r\n<br>Model Saem is 175cm tall, a UK6/8, wears Size S.\r\nModel A is 170cm tall, a UK6/8, wears Size S.\r\n"
4,https://www.lovebonito.com/sg/ariah-balloon-sleeve-top.html,200,OK,Ariah Balloon Sleeve Top,S$ 41.90,NaN,<br>100% Polyester,"Flair for the dramatic. Ruffle hem. Relaxed fit. Keyhole back.<br>Model A is 170cm tall, a UK6/8, wears Size S.\r\n"


In [484]:
lb.isnull().sum()

Address               0
Status Code           0
Status                0
title 1               0
price 1               3
colour 1           1893
material 1          957
Description 1 1     934
dtype: int64

In [485]:
lb['colour 1'] = 'missing'

In [486]:
lb['material 1'] = lb['material 1'].str.replace("<br>","")

In [487]:
lb.rename(columns={'colour 1':'colour', 'material 1':'material', 'price 1':'price', 'Description 1 1':'description', 'title 1':'title' },inplace=True)

In [488]:
lb['price'] = lb['price'].astype(str)
lb['price']=[x[3:] for x in lb.price]

In [489]:
lb['material'].fillna("missing",inplace=True)

In [490]:
lb.head()

,Address,Status Code,Status,title,price,colour,material,description
0,https://www.lovebonito.com/sg/meisel-shawl-collar-blazer.html,200,OK,Meisel Shawl Collar Blazer,51.90,missing,100% Polyester,"Suit up, ladies. Gathered sleeve detail. Open front. <br>Model Cindy is 178cm tall, a UK6/8, wears Size S.\r\n"
1,https://www.lovebonito.com/sg/blaise-midi-polka-dot-dress.html,200,OK,Blaise Midi Polka Dot Dress,51.90,missing,100% Polyester,"Flair for the dramatic. Bold polkadot print. Ruched waist. Puff sleeves.<br>Model MM is 176cm tall, a UK6/8, wears Size S.\r\n"
2,https://www.lovebonito.com/sg/marley-printed-eco-bag-in-floral-waltz.html,200,OK,Marley Printed Eco Bag in Floral Waltz,14.90,missing,missing,<b>Specially designed in-house print.</b> Responsibly chic. Twin handles.<br>
3,https://www.lovebonito.com/sg/jeryl-ribbon-tie-top.html,200,OK,Jeryl Ribbon Tie Top,36.90,missing,100% Polyester,"For your day-time thing. Ribbon tie straps. Button details. V-neckline. \r\n<br>Model Saem is 175cm tall, a UK6/8, wears Size S.\r\nModel A is 170cm tall, a UK6/8, wears Size S.\r\n"
4,https://www.lovebonito.com/sg/ariah-balloon-sleeve-top.html,200,OK,Ariah Balloon Sleeve Top,41.90,missing,100% Polyester,"Flair for the dramatic. Ruffle hem. Relaxed fit. Keyhole back.<br>Model A is 170cm tall, a UK6/8, wears Size S.\r\n"


In [491]:
lb['category'] = 0
for row in range(lb.shape[0]):
    if "Top" in lb.loc[row,'title']:
        lb.loc[row,'category'] = "tops"
    elif "Dress" in lb.loc[row,'title']:
        lb.loc[row,'category'] = "dress"
    elif "Shorts" in lb.loc[row,'title']:
        lb.loc[row,'category'] = "shorts"
    elif "Skirt" in lb.loc[row,'title']:
        lb.loc[row,'category'] = "skirts"
    elif "Shirt" in lb.loc[row,'title']:
        lb.loc[row,'category'] = "tops"
    elif "Jumpsuit" in lb.loc[row,'title']:
        lb.loc[row,'category'] = "jumpsuits"
    elif "Pants" in lb.loc[row,'title']:
        lb.loc[row,'category'] = "bottoms"
    elif "Blouse" in lb.loc[row,'title']:
        lb.loc[row,'category'] = "tops"
    elif "Culottes" in lb.loc[row,'title']:
        lb.loc[row,'category'] = "bottoms"
    elif "Tee" in lb.loc[row,'title']:
        lb.loc[row,'category'] = "tops"
    elif "Kimono" in lb.loc[row,'title']:
        lb.loc[row,'category'] = "outerwear"
    elif "Kaftan" in lb.loc[row,'title']:
        lb.loc[row,'category'] = "outerwear"
    elif "Blazer" in lb.loc[row,'title']:
        lb.loc[row,'category'] = "outerwear"
    elif "Vest" in lb.loc[row,'title']:
        lb.loc[row,'category'] = "outerwear"
    elif "Cami" in lb.loc[row,'title']:
        lb.loc[row,'category'] = "tops"
    elif "Sweater" in lb.loc[row,'title']:
        lb.loc[row,'category'] = "outerwear"
    elif "Jacket" in lb.loc[row,'title']:
        lb.loc[row,'category'] = "outerwear"
    elif "Cardigan" in lb.loc[row,'title']:
        lb.loc[row,'category'] = "outerwear"
    elif "Skort" in lb.loc[row,'title']:
        lb.loc[row,'category'] = "shorts"
    elif "T-shirt" in lb.loc[row,'title']:
        lb.loc[row,'category'] = "tops"
    elif "Playsuit" in lb.loc[row,'title']:
        lb.loc[row,'category'] = "jumpsuit"
    elif "Romper" in lb.loc[row,'title']:
        lb.loc[row,'category'] = "jumpsuit"
    elif "Jean" in lb.loc[row,'title']:
        lb.loc[row,'category'] = "jeans"
    elif "Tank" in lb.loc[row,'title']:
        lb.loc[row,'category'] = "tops"
    elif "Miniskirt" in lb.loc[row,'title']:
        lb.loc[row,'category'] = "skirts"


In [492]:
for row in range(lb.shape[0]):
    if lb.loc[row,'category'] == 0:
#         print(lb.loc[row,'title'])
        lb.drop(row,inplace=True)

In [493]:
lb.shape

(1727, 9)

In [494]:
lb = lb.reset_index(drop=True)

In [495]:
for row in range(lb.shape[0]): 
    lb['description'] = lb['description'].astype(str)
    lb.loc[row, 'description'] = lb.loc[row,'description'].split("<br>")[0]
    lb.loc[row,'description'] = lb.loc[row,'description'].replace("\r","")
    lb.loc[row,'description'] = lb.loc[row,'description'].replace("\n","")

In [504]:
lb.tail()

,title,price,colour,material,category,description,brand
1694,Freja Layered Midi Dress,51.9,missing,missing,dress,Freja Layered Midi Dress,Love Bonito
1695,Ardelo Printed Ruffle Shoulder Pencil Dress,46.9,missing,missing,dress,Ardelo Printed Ruffle Shoulder Pencil Dress,Love Bonito
1696,Ardelo Printed Ruffle Shoulder Pencil Dress,46.9,missing,missing,dress,Ardelo Printed Ruffle Shoulder Pencil Dress,Love Bonito
1697,Myrna Striped Cross Back Midi Dress,46.9,missing,100% Polyester,dress,"We’re taking the dress with us on a picnic. Features the on-trend square neckline, ruffle hem and slimming stripes. Comes with optional ribbon tie to cinch the dress in at the waist.. Concealed side zip closure with hook. - Comes with detachable sash.",Love Bonito
1698,Covet Cadwyne Tuxedo Dress,48.9,missing,missing,dress,Covet Cadwyne Tuxedo Dress,Love Bonito


In [497]:
lb['price'] = lb['price'].astype(float)
lb= lb[lb['price']>0]
lb = lb.reset_index(drop=True)

In [498]:
lb.dtypes

Address         object
Status Code      int64
Status          object
title           object
price          float64
colour          object
material        object
description     object
category        object
dtype: object

In [499]:
features = ['title','price','colour','material','category','description']
lb=lb[features]
lb['brand'] = 'Love Bonito'

In [501]:
lb.isnull().sum()

title          0
price          0
colour         0
material       0
category       0
description    0
brand          0
dtype: int64

In [508]:
for row in range(lb.shape[0]):
    if lb.loc[row,'description'] == 'nan':
        lb.loc[row,'description'] = lb.loc[row,'title']
        lb.loc[row,'description'] = lb.loc[row,'description'] + " " + lb.loc[row,'material']
lb.drop(columns=['material'],inplace=True)

In [509]:
lb.to_csv("lb_clean.csv")

## ASOS (Jeans)

In [452]:
asos_jeans1 = pd.read_csv('../datasets/asos_jeans1.csv')
asos_jeans2 = pd.read_csv('../datasets/asos_jeans2.csv')
asos_links = pd.read_csv('../Scrapers/links/asos_jeans.csv')

In [453]:
asos_jeans = pd.concat([asos_jeans1, asos_jeans2])

In [454]:
asos_jeans.drop(columns=['Unnamed: 0'],inplace=True)
asos_jeans = asos_jeans.reset_index(drop=True)

In [455]:
asos_jeans.shape

(1476, 6)

In [456]:
asos_links.shape

(1476, 4)

In [457]:
asos_links.head()

,web-scraper-order,web-scraper-start-url,link,link-href
0,1575397378-49381,https://www.asos.com/women/jeans/cat/?cid=3630&nlid=ww|clothing|shop+by+product,Pimkie acid mom jean in blueSGD$39.56,https://www.asos.com/pimkie/pimkie-acid-mom-jean-in-blue/prd/13540425?clr=blue&colourWayId=16545163&SearchQuery=&cid=3630
1,1575397378-48964,https://www.asos.com/women/jeans/cat/?cid=3630&nlid=ww|clothing|shop+by+product,Reclaimed Vintage The '90 skinny jeans in washed blackSGD$53.97,https://www.asos.com/reclaimed-vintage/reclaimed-vintage-the-90-skinny-jeans-in-washed-black/prd/10691256?clr=washed-black&colourWayId=16292267&SearchQuery=&cid=3630
2,1575397378-49193,https://www.asos.com/women/jeans/cat/?cid=3630&nlid=ww|clothing|shop+by+product,COLLUSION x005 straight leg jeans in vintage washSGD$35.98,https://www.asos.com/collusion/collusion-x005-straight-leg-jeans-in-vintage-wash/prd/11626001?clr=blue&colourWayId=16383296&SearchQuery=&cid=3630
3,1575397378-48639,https://www.asos.com/women/jeans/cat/?cid=3630&nlid=ww|clothing|shop+by+product,Missguided riot mom jeans in blackSGD$44.98,https://www.asos.com/missguided/missguided-riot-mom-jeans-in-black/prd/13384825?clr=black&colourWayId=16465807&SearchQuery=&cid=3630
4,1575397378-49025,https://www.asos.com/women/jeans/cat/?cid=3630&nlid=ww|clothing|shop+by+product,ASOS DESIGN Lisbon mid rise skinny jeans in clean black in ankle grazer lengthSGD$39.58,https://www.asos.com/asos-design/asos-design-lisbon-mid-rise-skinny-jeans-in-clean-black-in-ankle-grazer-length/prd/8552875?clr=clean-black&colourWayId=15194773&SearchQuery=&cid=3630


In [458]:
df_newcols = {
    'link-href':'url'
}
asos_links.rename(columns=df_newcols,inplace=True)

In [461]:
asos_jeans.head()

,url,brand,title,price,description,colour
0,https://www.asos.com/pimkie/pimkie-acid-mom-jean-in-blue/prd/13540425?clr=blue&colourWayId=16545163&SearchQuery=&cid=3630,Pimkie,Pimkie acid mom jean in blue,SGD$39.56,It's all in the jeans High rise Belt loops Concealed fly Five pockets Straight cut Relaxed fit Think baggy and casual,Blue
1,https://www.asos.com/reclaimed-vintage/reclaimed-vintage-the-90-skinny-jeans-in-washed-black/prd/10691256?clr=washed-black&colourWayId=16292267&SearchQuery=&cid=3630,Reclaimed Vintage,Reclaimed Vintage The '90 skinny jeans in washed black,SGD$53.97,You can always depend on denim Zip fly Functional pockets Skinny fit A tight cut for a sleek shape Exclusive to ASOS,Washed black
2,https://www.asos.com/collusion/collusion-x005-straight-leg-jeans-in-vintage-wash/prd/11626001?clr=blue&colourWayId=16383296&SearchQuery=&cid=3630,COLLUSION,COLLUSION x005 straight leg jeans in vintage wash,SGD$35.98,They're long-term relationship material Light wash High rise Concealed fly with button closure Functioning pockets Regular fit Just select your usual size,Blue
3,https://www.asos.com/missguided/missguided-riot-mom-jeans-in-black/prd/13384825?clr=black&colourWayId=16465807&SearchQuery=&cid=3630,Missguided,Missguided riot mom jeans in black,SGD$44.98,They're long-term relationship material High rise Concealed fly Functional pockets Cropped length Regular fit Just select your usual size,Black
4,https://www.asos.com/asos-design/asos-design-lisbon-mid-rise-skinny-jeans-in-clean-black-in-ankle-grazer-length/prd/8552875?clr=clean-black&colourWayId=15194773&SearchQuery=&cid=3630,skinny jeans,ASOS DESIGN Lisbon mid rise skinny jeans in clean black in ankle grazer length,SGD$39.58,They're long-term relationship material Mid-rise waist Zip fly Belt loops Five pockets Skinny fit Cut closely from hips to hem,Clean black


In [462]:
asos_jeans['description'] = asos_jeans['description'].str.replace("<li>"," ")
asos_jeans['description'] = asos_jeans['description'].str.replace("</li>"," ")
asos_jeans['description'] = asos_jeans['description'].str.replace('<span style="background-color: initial;">'," ")
asos_jeans['description'] = asos_jeans['description'].str.replace("<span>"," ")
asos_jeans['description'] = asos_jeans['description'].str.replace("&nbsp;"," ")
asos_jeans['description'] = asos_jeans['description'].str.replace("</span>"," ")

In [463]:
asos_jeans = asos_jeans.merge(asos_links, on='url', how="left",  left_index=True)

In [464]:
asos_jeans['category'] = "jeans"

In [465]:
asos_jeans['price'] = asos_jeans['price'].astype(str)
asos_jeans['price']=[x[4:] for x in asos_jeans.price]


In [466]:
asos_jeans = asos_jeans.reset_index(drop=True)

In [467]:
for row in range(asos_jeans.shape[0]):
    if asos_jeans.loc[row, 'price'] == 'ing':
        asos_jeans.loc[row, 'price'] = asos_jeans.loc[row,'link'].split("SGD$")[1]
#     else:
#         asos_jeans.loc[row, 'price'] = asos_jeans.loc[row,'price'].split("SGD$")[1]
        
         

In [468]:
asos_jeans.shape

(1476, 10)

In [469]:
tofix = asos_jeans[asos_jeans.loc[:,'description'] == 'missing']
tofix

,url,brand,title,price,description,colour,web-scraper-order,web-scraper-start-url,link,category
192,https://www.asos.com/one-above-another/one-above-another-oversized-jacket-straight-leg-jeans-in-tie-dye-denim-co-ord/grp/25152?clr=grunge-tie-dye&colourWayId=16456345&SearchQuery=&cid=3630,COLLUSION,One Above Another oversized jacket & straight leg jeans in tie dye denim co-ord,80.96,missing,Grunge tie dye,1575397378-49205,https://www.asos.com/women/jeans/cat/?cid=3630&nlid=ww|clothing|shop+by+product,MIX & MATCHOne Above Another oversized jacket & straight leg jeans in tie dye denim co-ordFrom SGD$152.92,jeans
695,https://www.asos.com/new-look-petite/new-look-petite-high-waisted-skinny-jean-in-black/prd/14229183?clr=black&colourWayId=16601000&SearchQuery=&cid=3630,ASOS DESIGN,New Look Petite high-waisted skinny jean in black,46.76,missing,missing,1575397378-49406,https://www.asos.com/women/jeans/cat/?cid=3630&nlid=ww|clothing|shop+by+product,New Look Petite high-waisted skinny jean in blackSGD$46.76,jeans
1370,https://www.asos.com/collusion/collusion-unisex-bleached-cord-jean-jacket/grp/26774?clr=khaki&colourWayId=0&SearchQuery=&cid=3630,Pimkie,COLLUSION Unisex bleached cord jean & jacket,125.93,missing,missing,1575397378-48828,https://www.asos.com/women/jeans/cat/?cid=3630&nlid=ww|clothing|shop+by+product,MIX & MATCHCOLLUSION Unisex bleached cord jean & jacketFrom SGD$125.93,jeans


In [472]:
asos_jeans.isnull().sum()

url                      0
brand                    0
title                    0
price                    0
description              0
colour                   0
web-scraper-order        0
web-scraper-start-url    0
link                     0
category                 0
dtype: int64

In [471]:
asos_jeans.fillna("a",inplace=True)
for row in range(asos_jeans.shape[0]):
    if asos_jeans.loc[row,'brand'] =='a':
        asos_jeans.loc[row,'brand'] = asos_jeans.loc[row,'url'].split("/")[3]

In [473]:
features = ['title','brand','price','description','colour','category']
asos_jeans = asos_jeans[features]

In [475]:
asos_jeans['brand'] = asos_jeans['brand'].str.replace("-"," ")

In [476]:
asos_jeans.to_csv("asos_jeans.csv")

## ASOS trousers

In [418]:
asos_trousers = pd.read_csv('../datasets/asos_trousers.csv')
asos_links = pd.read_csv('../Scrapers/links/asos_trousers.csv')

In [419]:
asos_trousers.shape

(3348, 7)

In [420]:
asos_links.shape

(3348, 4)

In [421]:
asos_trousers.columns

Index(['Unnamed: 0', 'url', 'brand', 'title', 'price', 'description',
       'colour'],
      dtype='object')

In [422]:
asos_links.columns

Index(['web-scraper-order', 'web-scraper-start-url', 'link', 'link-href'], dtype='object')

In [423]:
asos_trousers.drop(columns=['Unnamed: 0'],inplace=True)
asos_trousers = asos_trousers.reset_index(drop=True)

In [424]:
df_newcols = {
    'link-href':'url'
}
asos_links.rename(columns=df_newcols,inplace=True)

In [447]:
asos_trousers.head()

,url,brand,title,price,description,colour,web-scraper-order,web-scraper-start-url,link
0,https://www.asos.com/asos-petite/asos-design-petite-lounge-jogger/prd/8788932?clr=black&colourWayId=15224247&SearchQuery=&cid=2640,asos-petite,ASOS DESIGN Petite lounge jogger,35.98,Drawstring waistband Welcome to the cosy life Side pockets Fitted cuffs Relaxed fit Think baggy and casual,Black,1575444823-1547,https://www.asos.com/women/trousers-leggings/cat/?cid=2640&nlid=ww|clothing|shop+by+product,ASOS DESIGN Petite lounge joggerSGD$35.98
1,https://www.asos.com/asos-design/asos-design-shell-jogger-in-navy-colour-block/prd/12201440?clr=navy&colourWayId=16423562&SearchQuery=&cid=2640,ASOS DESIGN,ASOS DESIGN shell jogger in navy colour block,57.57,Throw on and go Stretch waistband Side pockets Fitted cuffs Regular fit Just pick your usual size,Navy,1575444823-2197,https://www.asos.com/women/trousers-leggings/cat/?cid=2640&nlid=ww|clothing|shop+by+product,MIX & MATCHASOS DESIGN shell jogger in navy colour blockSGD$57.57
2,https://www.asos.com/asos-4505/asos-4505-neon-colourblock-legging-with-bum-sculpt-seam-detail/prd/12830495?clr=black-neon-yellow&colourWayId=16522991&SearchQuery=&cid=2640,ASOS 4505,ASOS 4505 neon colourblock legging with bum sculpt seam detail,39.58,"Activewear, but make it fashion Colour-block design High rise Elasticated waistband Sculpting design 4505 logo to reverse Bodycon fit A tight cut to the body",Black/neon yellow,1575444823-507,https://www.asos.com/women/trousers-leggings/cat/?cid=2640&nlid=ww|clothing|shop+by+product,ASOS 4505 neon colourblock legging with bum sculpt seam detailSGD$39.58
3,https://www.asos.com/puma/puma-iridescent-joggers/prd/13554928?clr=multi&colourWayId=16546572&SearchQuery=&cid=2640,PUMA,Puma Iridescent Joggers,107.95,For the gym or your everyday thing Drawstring waistband Side pockets PUMA logo Fitted cuffs Regular fit True to size,missing,1575444823-267,https://www.asos.com/women/trousers-leggings/cat/?cid=2640&nlid=ww|clothing|shop+by+product,Puma Iridescent JoggersSGD$107.95
4,https://www.asos.com/tommy-jeans/tommy-jeans-organic-jogger-pant/prd/13034584?clr=tommy-black&colourWayId=16477320&SearchQuery=&cid=2640,Tommy Jeans Capsule,Tommy Jeans organic jogger pant,116.94,Part of our responsible edit High rise Drawstring waist Functional pockets Fitted cuffs Slim fit Cut close to the body,Tommy black,1575444823-865,https://www.asos.com/women/trousers-leggings/cat/?cid=2640&nlid=ww|clothing|shop+by+product,Tommy Jeans organic jogger pantSGD$116.94


In [426]:
asos_trousers['description'] = asos_trousers['description'].str.replace("<li>"," ")
asos_trousers['description'] = asos_trousers['description'].str.replace("</li>"," ")
asos_trousers['description'] = asos_trousers['description'].str.replace('<span style="background-color: initial;">'," ")
asos_trousers['description'] = asos_trousers['description'].str.replace("<span>"," ")
asos_trousers['description'] = asos_trousers['description'].str.replace("&nbsp;"," ")
asos_trousers['description'] = asos_trousers['description'].str.replace("</span>"," ")

In [427]:
asos_trousers = asos_trousers.merge(asos_links, on='url', how="left",  left_index=True)

In [451]:
asos_trousers.head()

,url,brand,title,price,description,colour,web-scraper-order,web-scraper-start-url,link
0,https://www.asos.com/asos-petite/asos-design-petite-lounge-jogger/prd/8788932?clr=black&colourWayId=15224247&SearchQuery=&cid=2640,asos petite,ASOS DESIGN Petite lounge jogger,35.98,Drawstring waistband Welcome to the cosy life Side pockets Fitted cuffs Relaxed fit Think baggy and casual,Black,1575444823-1547,https://www.asos.com/women/trousers-leggings/cat/?cid=2640&nlid=ww|clothing|shop+by+product,ASOS DESIGN Petite lounge joggerSGD$35.98
1,https://www.asos.com/asos-design/asos-design-shell-jogger-in-navy-colour-block/prd/12201440?clr=navy&colourWayId=16423562&SearchQuery=&cid=2640,ASOS DESIGN,ASOS DESIGN shell jogger in navy colour block,57.57,Throw on and go Stretch waistband Side pockets Fitted cuffs Regular fit Just pick your usual size,Navy,1575444823-2197,https://www.asos.com/women/trousers-leggings/cat/?cid=2640&nlid=ww|clothing|shop+by+product,MIX & MATCHASOS DESIGN shell jogger in navy colour blockSGD$57.57
2,https://www.asos.com/asos-4505/asos-4505-neon-colourblock-legging-with-bum-sculpt-seam-detail/prd/12830495?clr=black-neon-yellow&colourWayId=16522991&SearchQuery=&cid=2640,ASOS 4505,ASOS 4505 neon colourblock legging with bum sculpt seam detail,39.58,"Activewear, but make it fashion Colour-block design High rise Elasticated waistband Sculpting design 4505 logo to reverse Bodycon fit A tight cut to the body",Black/neon yellow,1575444823-507,https://www.asos.com/women/trousers-leggings/cat/?cid=2640&nlid=ww|clothing|shop+by+product,ASOS 4505 neon colourblock legging with bum sculpt seam detailSGD$39.58
3,https://www.asos.com/puma/puma-iridescent-joggers/prd/13554928?clr=multi&colourWayId=16546572&SearchQuery=&cid=2640,PUMA,Puma Iridescent Joggers,107.95,For the gym or your everyday thing Drawstring waistband Side pockets PUMA logo Fitted cuffs Regular fit True to size,missing,1575444823-267,https://www.asos.com/women/trousers-leggings/cat/?cid=2640&nlid=ww|clothing|shop+by+product,Puma Iridescent JoggersSGD$107.95
4,https://www.asos.com/tommy-jeans/tommy-jeans-organic-jogger-pant/prd/13034584?clr=tommy-black&colourWayId=16477320&SearchQuery=&cid=2640,Tommy Jeans Capsule,Tommy Jeans organic jogger pant,116.94,Part of our responsible edit High rise Drawstring waist Functional pockets Fitted cuffs Slim fit Cut close to the body,Tommy black,1575444823-865,https://www.asos.com/women/trousers-leggings/cat/?cid=2640&nlid=ww|clothing|shop+by+product,Tommy Jeans organic jogger pantSGD$116.94


In [429]:
asos_trousers = asos_trousers.reset_index(drop=True)

In [430]:
asos_trousers['price'] = asos_trousers['price'].astype(str)
asos_trousers['price']=[x[4:] for x in asos_trousers.price]

In [431]:
for row in range(asos_trousers.shape[0]):
    if asos_trousers.loc[row, 'price'] == 'ing':
        asos_trousers.loc[row, 'price'] = asos_trousers.loc[row,'link'].split("SGD$")[1]

In [432]:
tofix = asos_trousers[asos_trousers.loc[:,'brand'] == 'missing']
tofix

,url,brand,title,price,description,colour,web-scraper-order,web-scraper-start-url,link


In [433]:
asos_trousers.isnull().sum()

url                       0
brand                    15
title                     0
price                     0
description               0
colour                    0
web-scraper-order         0
web-scraper-start-url     0
link                      0
dtype: int64

In [434]:
url = "https://www.asos.com/tommy-jeans/tommy-jeans-organic-jogger-pant/prd/13034584?clr=tommy-black&colourWayId=16477320&SearchQuery=&cid=2640"

In [435]:
url.split("/")[3]

'tommy-jeans'

In [416]:
asos_trousers = asos_trousers.astype(str)

In [444]:
asos_trousers.fillna("a",inplace=True)
for row in range(asos_trousers.shape[0]):
    if asos_trousers.loc[row,'brand'] =='a':
        asos_trousers.loc[row,'brand'] = asos_trousers.loc[row,'url'].split("/")[3]

In [450]:
asos_trousers['brand'] = asos_trousers['brand'].str.replace("-"," ")

In [478]:
asos_trousers['category'] = "bottoms"

In [479]:
asos_trousers = asos_trousers[features]

In [483]:
asos_trousers['price'] = asos_trousers['price'].str.replace(" From ","")

/Users/lynn/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [484]:
asos_trousers.head()

,title,brand,price,description,colour,category
0,ASOS DESIGN Petite lounge jogger,asos petite,35.98,Drawstring waistband Welcome to the cosy life Side pockets Fitted cuffs Relaxed fit Think baggy and casual,Black,bottoms
1,ASOS DESIGN shell jogger in navy colour block,ASOS DESIGN,57.57,Throw on and go Stretch waistband Side pockets Fitted cuffs Regular fit Just pick your usual size,Navy,bottoms
2,ASOS 4505 neon colourblock legging with bum sculpt seam detail,ASOS 4505,39.58,"Activewear, but make it fashion Colour-block design High rise Elasticated waistband Sculpting design 4505 logo to reverse Bodycon fit A tight cut to the body",Black/neon yellow,bottoms
3,Puma Iridescent Joggers,PUMA,107.95,For the gym or your everyday thing Drawstring waistband Side pockets PUMA logo Fitted cuffs Regular fit True to size,missing,bottoms
4,Tommy Jeans organic jogger pant,Tommy Jeans Capsule,116.94,Part of our responsible edit High rise Drawstring waist Functional pockets Fitted cuffs Slim fit Cut close to the body,Tommy black,bottoms


In [485]:
asos_trousers.to_csv("asos_trousers.csv")

In [486]:
asos_trousers.price.unique()

array(['35.98', '57.57', '39.58', '107.95', '116.94', '64.77', '35.96',
       '68.37', '53.97', '61.17', '125.94', '62.97', '80.96', '70.16',
       '44.98', '89.96', '71.96', '86.36', '53.96', '187.11', '41.36',
       '156.52', '98.95 ', '25.19', '50.37 ', '32.38', '17.97', '50.37',
       '26.99', '98.95', '151.12', '75.56', '30.57', '50.36', '26.97',
       '21.59', '183.50', '19.77', '26.99 ', '143.93', '68.36', '57.57 ',
       '28.79', '46.76', '59.35', '350.82', '52.17', '152.92', '43.18',
       '32.37', '51.27', '138.53', '449.78', '134.93', '413.79', '34.18',
       '88.16', '124.14', '35.98 ', '14.39', '53.96 ', '93.55', '39.56',
       '95.35', '79.16', '82.76', '179.91', '44.98 ', '260.87', '53.98',
       '323.84', '356.22', '48.58', '62.97 ', '80.96 ', '197.90',
       '305.85', '53.97 ', '111.54', '17.99', '100.75', '100.74', '46.78',
       '97.15', '64.75', '170.91', '28.77', '59.33', '165.51', '68.37 ',
       '449.77', '125.33', '77.36', '143.84', '27.89', '190.70

## ASOS Lingerie

In [488]:
asos_lingerie = pd.read_csv('../datasets/asos_lingerie.csv')
asos_links = pd.read_csv('../Scrapers/links/asos_lingerie.csv')

In [489]:
asos_lingerie.shape

(1906, 7)

In [490]:
asos_links.shape

(1906, 4)

In [491]:
asos_lingerie.drop(columns=['Unnamed: 0'],inplace=True)
asos_lingerie = asos_lingerie.reset_index(drop=True)

In [492]:
df_newcols = {
    'link-href':'url'
}
asos_links.rename(columns=df_newcols,inplace=True)

In [493]:
asos_links.head()

,web-scraper-order,web-scraper-start-url,link,url
0,1575451960-4780,https://www.asos.com/women/lingerie-nightwear/cat/?cid=6046&nlid=ww|clothing|shop+by+product,Lindex eco viscose soft revere pyjama bottom in blackSGD$26.99,https://www.asos.com/lindex/lindex-eco-viscose-soft-revere-pyjama-bottom-in-black/prd/13134325?clr=black&colourWayId=16448115&SearchQuery=&cid=6046
1,1575451960-4164,https://www.asos.com/women/lingerie-nightwear/cat/?cid=6046&nlid=ww|clothing|shop+by+product,MIX & MATCHASOS DESIGN Odeda metallic rust cut out underboob crop braSGD$28.79,https://www.asos.com/asos-design/asos-design-odeda-metallic-rust-cut-out-underboob-crop-bra/prd/11681175?clr=rust&colourWayId=16391527&SearchQuery=&cid=6046
2,1575451960-3808,https://www.asos.com/women/lingerie-nightwear/cat/?cid=6046&nlid=ww|clothing|shop+by+product,City Chic Smooth & Chic Control BriefSGD$50.37 SGD$39.58-21%,https://www.asos.com/city-chic/city-chic-smooth-chic-control-brief/prd/8230465?clr=latte&colourWayId=15150134&SearchQuery=&cid=6046
3,1575451960-3927,https://www.asos.com/women/lingerie-nightwear/cat/?cid=6046&nlid=ww|clothing|shop+by+product,ASOS DESIGN Petite mix & match pyjama trouserSGD$21.59,https://www.asos.com/asos-petite/asos-design-petite-mix-match-pyjama-trouser/prd/12203742?clr=coral&colourWayId=16409967&SearchQuery=&cid=6046
4,1575451960-3617,https://www.asos.com/women/lingerie-nightwear/cat/?cid=6046&nlid=ww|clothing|shop+by+product,MIX & MATCHASOS DESIGN mix & match jersey trouser with satin detailSGD$25.19,https://www.asos.com/asos-design/asos-design-mix-match-jersey-trouser-with-satin-detail/prd/12553343?clr=lilac&colourWayId=16438640&SearchQuery=&cid=6046


In [496]:
asos_lingerie.head()

,url,brand,title,price,description,colour
0,https://www.asos.com/lindex/lindex-eco-viscose-soft-revere-pyjama-bottom-in-black/prd/13134325?clr=black&colourWayId=16448115&SearchQuery=&cid=6046,Lindex,Lindex eco viscose soft revere pyjama bottom in black,missing,You're invited to this PJ party Stretch waist Contrast piping Regular fit Just select your usual size,missing
1,https://www.asos.com/asos-design/asos-design-odeda-metallic-rust-cut-out-underboob-crop-bra/prd/11681175?clr=rust&colourWayId=16391527&SearchQuery=&cid=6046,ASOS DESIGN,ASOS DESIGN Odeda metallic rust cut out underboob crop bra,missing,For those ultra-fem vibes Square neck Cut-out design Adjustable straps Hook closure,missing
2,https://www.asos.com/city-chic/city-chic-smooth-chic-control-brief/prd/8230465?clr=latte&colourWayId=15150134&SearchQuery=&cid=6046,City Chic,City Chic Smooth & Chic Control Brief,missing,"Smooth stretch fabric High rise Control panel Gives a streamline silhouette Full-cut brief Wipe wit a damp cloth 76% Nylon, 24% Elastane",missing
3,https://www.asos.com/asos-petite/asos-design-petite-mix-match-pyjama-trouser/prd/12203742?clr=coral&colourWayId=16409967&SearchQuery=&cid=6046,ASOS DESIGN,ASOS DESIGN Petite mix & match pyjama trouser,missing,Part of our responsible edit Drawstring waist Wide-cut leg Regular fit Just select your usual size,missing
4,https://www.asos.com/asos-design/asos-design-mix-match-jersey-trouser-with-satin-detail/prd/12553343?clr=lilac&colourWayId=16438640&SearchQuery=&cid=6046,ASOS DESIGN,ASOS DESIGN mix & match jersey trouser with satin detail,missing,"Part of our responsible edit For when plans get cancelled, but you weren't going anyway Drawstring waist with satin-style tie Wide leg Relaxed fit For a slouchy silhouette",missing


In [495]:
asos_lingerie['description'] = asos_lingerie['description'].str.replace("<li>"," ")
asos_lingerie['description'] = asos_lingerie['description'].str.replace("</li>"," ")
asos_lingerie['description'] = asos_lingerie['description'].str.replace('<span style="background-color: initial;">'," ")
asos_lingerie['description'] = asos_lingerie['description'].str.replace("<span>"," ")
asos_lingerie['description'] = asos_lingerie['description'].str.replace("&nbsp;"," ")
asos_lingerie['description'] = asos_lingerie['description'].str.replace("</span>"," ")

In [497]:
asos_lingerie = asos_lingerie.merge(asos_links, on='url', how="left",  left_index=True)

In [502]:
asos_lingerie.head()

,url,brand,title,price,description,colour,web-scraper-order,web-scraper-start-url,link
0,https://www.asos.com/lindex/lindex-eco-viscose-soft-revere-pyjama-bottom-in-black/prd/13134325?clr=black&colourWayId=16448115&SearchQuery=&cid=6046,Lindex,Lindex eco viscose soft revere pyjama bottom in black,26.99,You're invited to this PJ party Stretch waist Contrast piping Regular fit Just select your usual size,missing,1575451960-4780,https://www.asos.com/women/lingerie-nightwear/cat/?cid=6046&nlid=ww|clothing|shop+by+product,Lindex eco viscose soft revere pyjama bottom in blackSGD$26.99
1,https://www.asos.com/asos-design/asos-design-odeda-metallic-rust-cut-out-underboob-crop-bra/prd/11681175?clr=rust&colourWayId=16391527&SearchQuery=&cid=6046,ASOS DESIGN,ASOS DESIGN Odeda metallic rust cut out underboob crop bra,28.79,For those ultra-fem vibes Square neck Cut-out design Adjustable straps Hook closure,missing,1575451960-4164,https://www.asos.com/women/lingerie-nightwear/cat/?cid=6046&nlid=ww|clothing|shop+by+product,MIX & MATCHASOS DESIGN Odeda metallic rust cut out underboob crop braSGD$28.79
2,https://www.asos.com/city-chic/city-chic-smooth-chic-control-brief/prd/8230465?clr=latte&colourWayId=15150134&SearchQuery=&cid=6046,City Chic,City Chic Smooth & Chic Control Brief,50.37,"Smooth stretch fabric High rise Control panel Gives a streamline silhouette Full-cut brief Wipe wit a damp cloth 76% Nylon, 24% Elastane",missing,1575451960-3808,https://www.asos.com/women/lingerie-nightwear/cat/?cid=6046&nlid=ww|clothing|shop+by+product,City Chic Smooth & Chic Control BriefSGD$50.37 SGD$39.58-21%
3,https://www.asos.com/asos-petite/asos-design-petite-mix-match-pyjama-trouser/prd/12203742?clr=coral&colourWayId=16409967&SearchQuery=&cid=6046,ASOS DESIGN,ASOS DESIGN Petite mix & match pyjama trouser,21.59,Part of our responsible edit Drawstring waist Wide-cut leg Regular fit Just select your usual size,missing,1575451960-3927,https://www.asos.com/women/lingerie-nightwear/cat/?cid=6046&nlid=ww|clothing|shop+by+product,ASOS DESIGN Petite mix & match pyjama trouserSGD$21.59
4,https://www.asos.com/asos-design/asos-design-mix-match-jersey-trouser-with-satin-detail/prd/12553343?clr=lilac&colourWayId=16438640&SearchQuery=&cid=6046,ASOS DESIGN,ASOS DESIGN mix & match jersey trouser with satin detail,25.19,"Part of our responsible edit For when plans get cancelled, but you weren't going anyway Drawstring waist with satin-style tie Wide leg Relaxed fit For a slouchy silhouette",missing,1575451960-3617,https://www.asos.com/women/lingerie-nightwear/cat/?cid=6046&nlid=ww|clothing|shop+by+product,MIX & MATCHASOS DESIGN mix & match jersey trouser with satin detailSGD$25.19


In [505]:
asos_lingerie.price.unique()

array(['26.99', '28.79', '50.37 ', '21.59', '25.19', '17.99', '14.39',
       '32.38', '46.78', '30.58', '59.37', '35.98', '34.18', '23.39',
       '10.79', '116.94', '52.17', '48.58', '53.97', '39.58', '46.77',
       '62.97', '57.57', '23.37', '68.37 ', '61.17', '64.76', '86.35',
       '89.96', '19.79', '44.98', '17.99 ', '50.37', '106.15', '50.38',
       '10.78', '75.57', '30.57', '68.37', '12.59', '13.49', '53.97 ',
       '79.16', '17.97', '75.56', '98.95', '64.77', '43.18', '105.25',
       '26.97', '107.95', '41.38', '71.96', '43.17', '9.00', '53.96',
       '12.58', '66.57', '161.92', '89.95', '31.48', '97.15', '237.48',
       '32.37', '47.68', '35.96', '76.46', '28.78', '70.16', '57.57 ',
       '14.37', '40.48', '32.38 ', '88.16', '21.59 ', '125.94', '16.17',
       '35.98 ', '77.36', '28.77', '44.98 ', '17.09', '73.77', '11.69',
       '62.95', '26.09 ', '64.77 ', '73.76', '134.93', '100.75', '48.58 ',
       '25.18', '14.39 ', '131.33', '35.08', '30.57 ', '46.78 ', '118.

In [500]:
asos_lingerie['price'] = asos_lingerie['price'].astype(str)
asos_lingerie['price']=[x[4:] for x in asos_lingerie.price]

In [501]:
for row in range(asos_lingerie.shape[0]):
    if asos_lingerie.loc[row, 'price'] == 'ing':
        asos_lingerie.loc[row, 'price'] = asos_lingerie.loc[row,'link'].split("SGD$")[1]

In [504]:
asos_lingerie['price'] = asos_lingerie['price'].str.replace(" From ","")

In [506]:
asos_lingerie.fillna("a",inplace=True)
for row in range(asos_lingerie.shape[0]):
    if asos_lingerie.loc[row,'brand'] =='a':
        asos_lingerie.loc[row,'brand'] = asos_lingerie.loc[row,'url'].split("/")[3]

In [507]:
asos_lingerie.isnull().sum()

url                      0
brand                    0
title                    0
price                    0
description              0
colour                   0
web-scraper-order        0
web-scraper-start-url    0
link                     0
dtype: int64

In [511]:
asos_lingerie['category'] = 'lingerie'

In [512]:
asos_lingerie= asos_lingerie[features]

In [513]:
asos_lingerie.to_csv("asos_lingerie.csv")